*Data Model Mapper*

The Data Model Mapper is a native application that allows providers to receive data from consumers in a standard format.

- The Provider defines the data they would like to receive from the consumer by defining target collections (tables, columns) and shares the app with a consumer
- The Consumer installs and uses the native app to model and map their data to the target collections, and shares the fully modeled data back to the provider

The native application includes a user interface designed for non-technical business users to allow them to map their data without the need for data engineers.

Let's start by setting up our environment with a dedicated role and warehouse

In [ ]:
/* cleanup */
use role accountadmin;
use warehouse xs_wh;
drop share if exists data_model_mapper_share;
drop application if exists data_model_mapper_app cascade;
alter listing if exists data_model_mapper_app set state = unpublished;
drop listing if exists data_model_mapper_app;
drop database if exists data_model_mapper_deployment_db;
drop database if exists data_model_mapper_app_package;
drop warehouse if exists data_model_mapper_wh;
drop database if exists dmm_test_db;
drop database if exists dmm_collection_metadata_db;
drop role if exists data_model_mapper_role;

In [ ]:

/* set up roles */
use role accountadmin;
call system$wait(5);

/* create role and add permissions required by role for installation of framework */
create role if not exists data_model_mapper_role;

/* perform grants */
set myusername = current_user();
grant role data_model_mapper_role to user identifier($myusername);
grant role data_model_mapper_role to role accountadmin;
/* with grant option needed to grant permissions to app */
grant create database on account to role data_model_mapper_role with grant option;
grant execute task on account to role data_model_mapper_role with grant option;

/* for adding the shares from the consumers */
grant import share on account to role data_model_mapper_role;

/* for creating the application package and listing */
grant create application package on account to role data_model_mapper_role;
grant create application on account to role data_model_mapper_role;
grant create data exchange listing on account to role data_model_mapper_role;
grant create share on account to role data_model_mapper_role;

/* for reading account metadata */
grant imported privileges on database snowflake to role data_model_mapper_role;

grant role data_model_mapper_role to role sysadmin;
--grant create warehouse on account to role data_model_mapper_role with grant option;

/* create warehouse */
create warehouse if not exists data_model_mapper_wh 
comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';

/* set role and warehouse */
use role data_model_mapper_role;
call system$wait(5);

Now let's create the application package and necessary tables

In [ ]:

/* let's create the app package - this defines the data content and application logic for an app, it is a provider-side object, and it not an installed instance of the actual app */
drop database if exists data_model_mapper_app_package;
create application package data_model_mapper_app_package
comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';

/* admin holds all base tables, managed by administrator */
create or replace schema data_model_mapper_app_package.admin 
comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';
drop schema if exists data_model_mapper_app_package.public;

/* customer list with Snowflake Org name */
create or replace table data_model_mapper_app_package.admin.customer (
    customer_name varchar,
    customer_snowflake_account_identifier varchar,
    customer_snowflake_organization_name varchar,
    last_updated_timestamp timestamp default current_timestamp(),
    constraint pkey primary key (customer_name) not enforced
) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';

/* validation rules set by provider */
/* Defining composite key for documentation and metadata. Snowflake doesn't enforce primary key constraints on tables because of the usage of micro-partitions. For more information, you can refer to: https://docs.snowflake.com/en/sql-reference/constraints */
create or replace table data_model_mapper_app_package.admin.validation_rule ( 
TARGET_TABLE VARCHAR NOT NULL,
RULE_NAME VARCHAR NOT NULL,
VALIDATION_TYPE VARCHAR NOT NULL,
VALIDATION_DESCRIP VARCHAR NOT NULL,
COLUMN_NAMES ARRAY NOT NULL,
OPERATOR VARCHAR, 
VALIDATION_PARAMS VARIANT,
CUSTOM_SQL_CONDITION VARCHAR,
PRIMARY KEY (TARGET_TABLE, RULE_NAME, VALIDATION_TYPE, COLUMN_NAMES)
);

/* Standard Validation Types: 
DOMAIN_OF_VALUES: Checks if the value of a column is within a specified set of values
IS_LENGTH: Checks if the length of a column is within a specified range
RANGE: Checks if the value of a column is within a specified range
NULL_OR_EMPTY: Checks if a column is null or empty
COLUMN_COMPARE: Checks if the value of one column is less than, equal to, or greater than another column
REQD_COLS_POPULATED: Checks if the required columns are populated 
COUNT: Checks the count of a column
REGEX: Checks if the value of a column matches a specified regular expression pattern
DATA_TYPE: Checks if the data type of a column is as expected
CUSTOM_SQL: Allows for a custom SQL validation to be written
*/

/* target collection - group of entities that define a data model, can be analogous to "product" */
create or replace table data_model_mapper_app_package.admin.target_collection (
    target_collection_name varchar,
    version varchar,
    last_updated_timestamp timestamp default current_timestamp(),
    constraint pkey primary key (target_collection_name, version) not enforced
) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';


/* customer target collection subscriptions - one customer to many target collections */
create or replace table data_model_mapper_app_package.admin.subscription (
    customer_name varchar,
    target_collection_name varchar,
    version varchar,
    expiration_date date,
    last_updated_timestamp timestamp default current_timestamp(),
    constraint pkey primary key (customer_name, target_collection_name) not enforced,
    constraint fkey_1 foreign key (customer_name) 
        references data_model_mapper_app_package.admin.customer (customer_name) not enforced,
    constraint fkey_2 foreign key (target_collection_name, version) 
        references data_model_mapper_app_package.admin.target_collection (target_collection_name, version) match partial not enforced
) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';


/* target entity - one target collection to many entities, can be analogous to "table" */
create or replace table data_model_mapper_app_package.admin.target_entity (
    target_collection_name varchar,
    version varchar,
    target_entity_name varchar,
    last_updated_timestamp timestamp default current_timestamp(),
    constraint pkey primary key (target_collection_name, version, target_entity_name) not enforced,
    constraint fkey_1 foreign key (target_collection_name, version) 
        references data_model_mapper_app_package.admin.target_collection (target_collection_name, version) not enforced
) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';


/* target entity attribute - one entity to many attributes, can be analogous to "column" */
create or replace table data_model_mapper_app_package.admin.target_entity_attribute (
    target_collection_name varchar,
    version varchar,
    target_entity_name varchar,
    target_entity_attribute_name varchar,
    target_attribute_properties object,
    last_updated_timestamp timestamp default current_timestamp(),
    constraint pkey primary key (target_collection_name, version, target_entity_name, target_entity_attribute_name) not enforced,
    constraint fkey_1 foreign key (target_collection_name, version, target_entity_name)
        references data_model_mapper_app_package.admin.target_entity (target_collection_name, version, target_entity_name) not enforced
) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';

/* grant read access to admin tables to the application package's built-in share */
grant usage on schema data_model_mapper_app_package.admin to share in application package data_model_mapper_app_package;
grant select on table data_model_mapper_app_package.admin.customer to share in application package data_model_mapper_app_package;
grant select on table data_model_mapper_app_package.admin.subscription to share in application package data_model_mapper_app_package;
grant select on table data_model_mapper_app_package.admin.target_collection to share in application package data_model_mapper_app_package;
grant select on table data_model_mapper_app_package.admin.target_entity to share in application package data_model_mapper_app_package;
grant select on table data_model_mapper_app_package.admin.target_entity_attribute to share in application package data_model_mapper_app_package;
grant select on table data_model_mapper_app_package.admin.validation_rule to share in application package data_model_mapper_app_package;

/* create the database and schema that will house our code for installation */
create or replace database dmm_test_db comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';
create or replace schema dmm_test_db.code comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';

Let's add some helper objects for deploying/redeploying the native app

These objects would typically be replaced by CI/CD tools in a production environment, but this keeps things simple

In [ ]:
create or replace database data_model_mapper_deployment_db comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';
create or replace schema data_model_mapper_deployment_db.code comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';
drop schema if exists data_model_mapper_deployment_db.public;
create or replace table data_model_mapper_deployment_db.code.file (
        NAME varchar
    ,   CONTENT varchar(16777216)
) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';

/* constructs and puts files into a stage */
CREATE OR REPLACE PROCEDURE data_model_mapper_deployment_db.code.PUT_TO_STAGE(STAGE VARCHAR,FILENAME VARCHAR, CONTENT VARCHAR)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION=3.8
PACKAGES=('snowflake-snowpark-python')
HANDLER='put_to_stage'
COMMENT='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
AS $$
import io
import os

def put_to_stage(session, stage, filename, content):
    local_path = '/tmp'
    local_file = os.path.join(local_path, filename)
    f = open(local_file, "w", encoding='utf-8')
    f.write(content)
    f.close()
    session.file.put(local_file, '@'+stage, auto_compress=False, overwrite=True)
    return "saved file "+filename+" in stage "+stage
$$;

/* helper stored procedure to help create image files */
create or replace procedure data_model_mapper_deployment_db.code.put_image_to_stage(stage varchar,filename varchar, base64_code varchar)
returns string
language python
runtime_version=3.8
packages=('snowflake-snowpark-python')
handler='put_image_to_stage'
COMMENT='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
AS $$
import io
import os
import base64


def put_image_to_stage(session, stage, filename, base64_code):
    local_path = '/tmp'
    local_file = os.path.join(local_path, filename)
    
    with open(local_file, "wb") as fw:
        img_data = base64_code.encode()
        fw.write(base64.b64decode(img_data))
        fw.close()
        session.file.put(local_file, '@'+stage, auto_compress=False, overwrite=True)
        return "saved file "+filename+" in stage "+stage
$$;

Now we will stage all of our app objects

In [ ]:
insert into data_model_mapper_deployment_db.code.file (NAME, CONTENT)
values ('MANIFEST',
$$
manifest_version: 1

version:
  name: Data Mapper
  label: “Data Mapper v1.0”
  comment: “Helps model and share data with the provider for collaboration purposes”

artifacts:
  readme: README.md
  setup_script: setup_script.sql
  default_streamlit: user_interface.data_modeler_streamlit
  extension_code: true

privileges:
- CREATE DATABASE:
    description: "To create the shareback database"
- CREATE WAREHOUSE:
    description: "To create the warehouse used for orchestration"
- EXECUTE TASK:
    description: "To manage data orchestration tasks"
$$
);

In [ ]:
insert into data_model_mapper_deployment_db.code.file SELECT 'SETUP' AS NAME, REGEXP_REPLACE($$
/*************************************************************************************************************
Script:             dmm Data Mapper - Native App - Setup Script v1
Create Date:        2023-10-25
Author:             B. Klein
Description:        Dynamic Data Model Mapper Native App -- Setup script that contains the objects that
                    the application will use when implemented on the consumer account.
Copyright © 2023 Snowflake Inc. All rights reserved
*************************************************************************************************************
SUMMARY OF CHANGES
Date(yyyy-mm-dd)    Author                              Comments
------------------- -------------------                 --------------------------------------------
2023-10-25          B. Klein                            Initial Creation
*************************************************************************************************************/

/*** make sure this file is ready, but do not run in a worksheet ***/

/* now, we need to build out what the customer interacts with to map their data to the target collection */

/* create application role for consumer access - we will use this later but want it created early */
create or replace application role dmm_consumer_app_role;

/* modeling holds all tables associated with defining a data model */
create or alter versioned schema modeling comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';

/* python procedure to initialize the shared database, based on application_name */
create or replace procedure modeling.initialize_application()
returns string
language python
runtime_version=3.8
packages=('snowflake-snowpark-python')
handler='initialize_application'
comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
as
:::
import snowflake.snowpark
from snowflake.snowpark.functions import sproc

def initialize_application(session):
    # get current database (app name)
    application_name = session.sql("""
        select current_database()
    """).collect()[0][0]
    
    # create warehouse
    session.sql("""
        create warehouse if not exists data_model_mapper_app_wh comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
    """).collect()

    # create database
    session.sql("""
        create or replace database DATA_MODEL_MAPPER_SHARE_DB comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
    """).collect()

    # create modeled schema for sharing mapped data
    session.sql("""
        create or replace schema DATA_MODEL_MAPPER_SHARE_DB.modeled comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
    """).collect()

    # create mapped schema for sharing mapped data
    session.sql("""
        create or replace schema DATA_MODEL_MAPPER_SHARE_DB.mapped comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
    """).collect()

    # create configuration schema for sharing app configuration
    session.sql("""
        create or replace schema DATA_MODEL_MAPPER_SHARE_DB.configuration comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
    """).collect()

    # create utility schema for creating helper stored procedures
    session.sql("""
        create or replace schema DATA_MODEL_MAPPER_SHARE_DB.utility comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
    """).collect()

    # create validated schema for sharing validation_log
    session.sql("""
        create or replace schema DATA_MODEL_MAPPER_SHARE_DB.validated comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
    """).collect()

    # drop unnecessary public schema
    session.sql("""
        drop schema if exists DATA_MODEL_MAPPER_SHARE_DB.public
    """).collect()

    # create configuration tables

    # source collection - group of entities that define a source data model
    # target_collection_name refers to the intended target collection for mapping
    # target_entity_name refers to the intended target entity for mapping
    # refresh frequency will be used the the orchestration feature later
    session.sql("""
    create table if not exists DATA_MODEL_MAPPER_SHARE_DB.configuration.source_collection (
        source_collection_name varchar,
        target_collection_name varchar,
        version varchar,
        target_entity_name varchar,
        custom_sql varchar,
        use_custom_sql boolean,
        generated_mapping_table varchar,
        refresh_frequency varchar,
        last_updated_timestamp timestamp default current_timestamp(),
        constraint pkey primary key (source_collection_name) not enforced
        ) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
    """).collect()

    # source entity with fully qualified origin - can be analogous to "source table"
    # note - only one source entity can be a base entity within a collection - all other entities are aggregated - it determines the cardinality of the collection
    # is_base_entity indicates that a source entity is the source entity to be joined on by other entities
    # join_from_source_entity_name specifies what existing source entity to join to
    # join_type should be typical SQL types - inner, left, right, etc.
    # join from is the left and join to is the right side of a join condition
    session.sql("""
    create table if not exists DATA_MODEL_MAPPER_SHARE_DB.configuration.source_entity (
        source_collection_name varchar,
        source_entity_name varchar,
        entity_fully_qualified_source varchar,
        is_base_entity boolean,
        join_from_source_entity_name varchar,
        join_type varchar,
        last_updated_timestamp timestamp default current_timestamp(),
        constraint pkey primary key (source_collection_name, source_entity_name) not enforced,
        constraint fkey_1 foreign key (source_collection_name) 
            references DATA_MODEL_MAPPER_SHARE_DB.configuration.source_collection (source_collection_name) not enforced
        ) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
    """).collect()

    # multiple join condition support, e.g. a.col1=b.col1 and a.col2=b.col2
    # join_from_entity_attribute_name - an attribute/column for the "from" side, "left" side of the join condition
    # operator - =, <, >, <=, >=
    # join_to_entity_attribute_name - an attribute/column for the "to" side, "right" side fo the join condition
    # note - multiple join conditions are combined with an AND
    session.sql("""
    create table if not exists DATA_MODEL_MAPPER_SHARE_DB.configuration.source_entity_join_condition (
        source_collection_name varchar,
        source_entity_name varchar,
        join_from_source_entity_name varchar,
        join_from_entity_attribute_name varchar,
        operator varchar,
        join_to_entity_attribute_name varchar,
        last_updated_timestamp timestamp default current_timestamp(),
        constraint pkey primary key (source_collection_name, source_entity_name, join_from_source_entity_name, operator, join_to_entity_attribute_name) not enforced,
        constraint fkey_1 foreign key (source_collection_name, source_entity_name) 
            references DATA_MODEL_MAPPER_SHARE_DB.configuration.source_entity (source_collection_name, source_entity_name) not enforced
        ) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
    """).collect()

    # source entity attribute - one entity to many attributes, can be analogous to "column"
    # note - aggregation will only be applied to non-base entity attributes
    # include_in_entity determines if a column is surfaced in the final table or not - allows for hiding irrelevant cols
    # derived_expression should be a scalar expression using attributes of the same entity, and null if not a derived col
    session.sql("""
    create table if not exists DATA_MODEL_MAPPER_SHARE_DB.configuration.source_entity_attribute (
        source_collection_name varchar,
        source_entity_name varchar,
        source_entity_attribute_name varchar,
        source_attribute_properties object,
        include_in_entity boolean,
        derived_expression varchar,
        aggregation_function varchar,
        last_updated_timestamp timestamp default current_timestamp(),
        constraint pkey primary key (source_collection_name, source_entity_name, source_entity_attribute_name) not enforced,
        constraint fkey_1 foreign key (source_collection_name, source_entity_name) 
            references DATA_MODEL_MAPPER_SHARE_DB.configuration.source_entity (source_collection_name, source_entity_name) not enforced
        ) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
    """).collect()

    # multiple filter condition support, e.g. col1=2000 and col2='ABC'
    # left_filter_expression - an expression for the "left" side of the filter condition
    # operator - =, <, >, <=, >=
    # right_filter_expression - an expression for the "right" side of the filter condition
    # note - multiple filter conditions are combined with an AND
    # applied AFTER the joins are performed
    session.sql("""
    create table if not exists DATA_MODEL_MAPPER_SHARE_DB.configuration.source_collection_filter_condition (
        source_collection_name varchar,
        left_filter_expression varchar,
        operator varchar,
        right_filter_expression varchar,
        last_updated_timestamp timestamp default current_timestamp(),
        constraint pkey primary key (source_collection_name, left_filter_expression, operator, right_filter_expression) not enforced,
        constraint fkey_1 foreign key (source_collection_name) 
            references DATA_MODEL_MAPPER_SHARE_DB.configuration.source_collection (source_collection_name) not enforced
        ) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
    """).collect()

    # defines the column mappings between source collection and target entity, populated by streamlit
    session.sql("""
    create table if not exists DATA_MODEL_MAPPER_SHARE_DB.configuration.source_to_target_mapping (
        source_collection_name varchar,
        generated_mapping_table_column_name varchar,
        target_attribute_name varchar,
        last_updated_timestamp timestamp default current_timestamp(),
        constraint pkey primary key (source_collection_name) not enforced,
        constraint fkey_1 foreign key (source_collection_name) 
            references DATA_MODEL_MAPPER_SHARE_DB.configuration.source_collection (source_collection_name) not enforced
        ) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
    """).collect()

    # create validation log table
    session.sql("""
    create table if not exists DATA_MODEL_MAPPER_SHARE_DB.validated.validation_log (
        RUN_TS TIMESTAMP_NTZ,
        TARGET_TABLE VARCHAR,
        RULE_NAME VARCHAR,
        VALIDATION_TYPE VARCHAR,
        VALIDATION_DESCRIP VARCHAR,
        COLUMN_NAMES ARRAY,
        OPERATOR VARCHAR,
        VALIDATION_PARAMS VARCHAR,
        CUSTOM_SQL_CONDITION VARCHAR,
        VALIDATION_RESULT VARCHAR,
        VALIDATION_NUM_PASSED INT,
        VALIDATION_NUM_FAILED INT,
        VALIDATION_MSG VARCHAR
        )
    """).collect()

    # grant application role permissions to created objects
    session.sql("""
        grant monitor on warehouse data_model_mapper_app_wh to application role dmm_consumer_app_role
    """.format(application_name=application_name)).collect()

    session.sql("""
        grant usage on database DATA_MODEL_MAPPER_SHARE_DB to application role dmm_consumer_app_role
    """.format(application_name=application_name)).collect()

    session.sql("""
        grant usage on schema DATA_MODEL_MAPPER_SHARE_DB.modeled to application role dmm_consumer_app_role
    """.format(application_name=application_name)).collect()

    session.sql("""
        grant usage on schema DATA_MODEL_MAPPER_SHARE_DB.mapped to application role dmm_consumer_app_role
    """.format(application_name=application_name)).collect()

    session.sql("""
        grant usage on schema DATA_MODEL_MAPPER_SHARE_DB.configuration to application role dmm_consumer_app_role
    """.format(application_name=application_name)).collect()

    session.sql("""
        grant usage on schema DATA_MODEL_MAPPER_SHARE_DB.utility to application role dmm_consumer_app_role
    """.format(application_name=application_name)).collect()

    session.sql("""
        grant usage on schema DATA_MODEL_MAPPER_SHARE_DB.validated to application role dmm_consumer_app_role
    """.format(application_name=application_name)).collect()

    session.sql("""
        grant select on table DATA_MODEL_MAPPER_SHARE_DB.configuration.source_collection to application role dmm_consumer_app_role with grant option
    """.format(application_name=application_name)).collect()

    session.sql("""
        grant select on table DATA_MODEL_MAPPER_SHARE_DB.configuration.source_collection_filter_condition to application role dmm_consumer_app_role with grant option
    """.format(application_name=application_name)).collect()

    session.sql("""
        grant select on table DATA_MODEL_MAPPER_SHARE_DB.configuration.source_entity to application role dmm_consumer_app_role with grant option
    """.format(application_name=application_name)).collect()

    session.sql("""
        grant select on table DATA_MODEL_MAPPER_SHARE_DB.configuration.source_entity_attribute to application role dmm_consumer_app_role with grant option
    """.format(application_name=application_name)).collect()

    session.sql("""
        grant select on table DATA_MODEL_MAPPER_SHARE_DB.configuration.source_entity_join_condition to application role dmm_consumer_app_role with grant option
    """.format(application_name=application_name)).collect()

    session.sql("""
        grant select on table DATA_MODEL_MAPPER_SHARE_DB.configuration.source_to_target_mapping to application role dmm_consumer_app_role with grant option
    """.format(application_name=application_name)).collect()

    session.sql("""
        grant select on table DATA_MODEL_MAPPER_SHARE_DB.validated.validation_log to application role dmm_consumer_app_role with grant option
    """.format(application_name=application_name)).collect()

    session.sql("""
        create or replace stage DATA_MODEL_MAPPER_SHARE_DB.utility.python_stg
    """.format(application_name=application_name)).collect()


    # create procedure for sharing views
    @sproc(name="DATA_MODEL_MAPPER_SHARE_DB.utility.share_views", is_permanent=True, stage_location="@DATA_MODEL_MAPPER_SHARE_DB.utility.python_stg", replace=True, packages=["snowflake-snowpark-python"])
    def share_views(session: snowflake.snowpark.Session) -> str:
        # get list of existing views
        session.sql("show views in DATA_MODEL_MAPPER_SHARE_DB.mapped").collect()
        view_df = session.sql('select "name" from table(result_scan(last_query_id()))')

        for view_row in view_df.to_local_iterator():
            share_sql_string = 'grant select on view DATA_MODEL_MAPPER_SHARE_DB.mapped."{view_name}" to share data_model_mapper_share'.format(view_name=view_row["name"])
            session.sql(share_sql_string).collect()

        return "Views shared successfully"

    # grant usage to application role
    session.sql("""
        grant usage on procedure DATA_MODEL_MAPPER_SHARE_DB.utility.share_views() to application role dmm_consumer_app_role with grant option
    """.format(application_name=application_name)).collect()

    return 'Shared database initialized successfully'
:::
;



/* python procedure to initially populate entity attributes */
create or replace procedure modeling.generate_attributes(source_collection_name varchar, source_entity_name varchar)
returns string
language python
runtime_version=3.8
packages=('snowflake-snowpark-python')
handler='generate_attributes'
comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
as
:::
from snowflake.snowpark.functions import call_udf, col, current_timestamp, when_matched, when_not_matched
from snowflake.snowpark.types import IntegerType, StringType, StructType, StructField

def generate_attributes(session, source_collection_name, source_entity_name):
    try:
        
        # entity dataframe
        entity_dataframe = session.table('DATA_MODEL_MAPPER_SHARE_DB.configuration.source_entity') \
            .filter((col('source_collection_name') == source_collection_name) & (col('source_entity_name') == source_entity_name))

        # gets the fully qualified source table
        source_table = str(entity_dataframe \
            .select(col('entity_fully_qualified_source')).distinct().collect()[0][0])

        # gets whether or not the entity is a base entity
        join_from_source_entity_name = str(entity_dataframe \
            .select(col('join_from_source_entity_name')).distinct().collect()[0][0])

        # generates show columns statement for source table
        show_col_sql_text = "show columns in table identifier('{source_table}')".format(source_table=source_table)

        session.sql(show_col_sql_text).collect()

        # use the last_query_id to get results of the show col statement
        last_query_results_sql_text = """
            select
                    '{source_collection_name}' as source_collection_name
                ,   '{source_entity_name}' as source_entity_name
                ,   "column_name" as source_entity_attribute_name
                ,   object_construct(
                        'data_type',parse_json("data_type"):type::varchar,
                        'is_nullable',parse_json("data_type"):nullable::boolean,
                        'precision',parse_json("data_type"):precision::number,
                        'scale',parse_json("data_type"):scale::number,
                        'length',parse_json("data_type"):length::number,
                        'byte_length',parse_json("data_type"):byteLength::number,
                        'description',null
                    ) as source_attribute_properties
                ,   TRUE as include_in_entity
                ,   NULL as derived_expression
                ,   case
                        when ('{join_from_source_entity_name}' <> 'None' and parse_json("data_type"):type::varchar = 'TEXT') then 'LISTAGG'
                        when ('{join_from_source_entity_name}' <> 'None' and parse_json("data_type"):type::varchar = 'FIXED') then 'SUM'
                        when ('{join_from_source_entity_name}' <> 'None' and parse_json("data_type"):type::varchar = 'BOOLEAN') then 'LISTAGG'
                        when ('{join_from_source_entity_name}' <> 'None' and parse_json("data_type"):type::varchar = 'DATE') then 'LISTAGG'
                        when ('{join_from_source_entity_name}' <> 'None' and parse_json("data_type"):type::varchar = 'TIMESTAMP_NTZ') then 'MAX'
                        when ('{join_from_source_entity_name}' <> 'None' and parse_json("data_type"):type::varchar = 'TIMESTAMP_LTZ') then 'MAX'
                        when ('{join_from_source_entity_name}' <> 'None' and parse_json("data_type"):type::varchar = 'TIMESTAMP_TZ') then 'MAX'
                        when ('{join_from_source_entity_name}' <> 'None') then 'MAX'
                        else null
                    end as aggregation_function
            from table(RESULT_SCAN(LAST_QUERY_ID()))
        """.format(source_collection_name=source_collection_name, source_entity_name=source_entity_name, join_from_source_entity_name=join_from_source_entity_name)

        source_df = session.sql(last_query_results_sql_text).with_column("LAST_UPDATED_TIMESTAMP", current_timestamp())

        target_df = session.table('DATA_MODEL_MAPPER_SHARE_DB.configuration.SOURCE_ENTITY_ATTRIBUTE')

        # merge with table on pk cols
        target_df.merge(
            source_df,
            (
                (target_df["SOURCE_COLLECTION_NAME"] == source_df["SOURCE_COLLECTION_NAME"]) &
                (target_df["SOURCE_ENTITY_NAME"] == source_df["SOURCE_ENTITY_NAME"]) &
                (target_df["SOURCE_ENTITY_ATTRIBUTE_NAME"] == source_df["SOURCE_ENTITY_ATTRIBUTE_NAME"])
            )
            ,
            [
                when_matched().update(
                    {
                        "SOURCE_ATTRIBUTE_PROPERTIES": source_df["SOURCE_ATTRIBUTE_PROPERTIES"],
                        "INCLUDE_IN_ENTITY": source_df["INCLUDE_IN_ENTITY"],
                        "DERIVED_EXPRESSION": source_df["DERIVED_EXPRESSION"],
                        "AGGREGATION_FUNCTION": source_df["AGGREGATION_FUNCTION"],
                        "LAST_UPDATED_TIMESTAMP": source_df["LAST_UPDATED_TIMESTAMP"]
                    }
                ),
                when_not_matched().insert(
                    {
                        "SOURCE_COLLECTION_NAME": source_df["SOURCE_COLLECTION_NAME"],
                        "SOURCE_ENTITY_NAME": source_df["SOURCE_ENTITY_NAME"],
                        "SOURCE_ENTITY_ATTRIBUTE_NAME": source_df["SOURCE_ENTITY_ATTRIBUTE_NAME"],
                        "SOURCE_ATTRIBUTE_PROPERTIES": source_df["SOURCE_ATTRIBUTE_PROPERTIES"],
                        "INCLUDE_IN_ENTITY": source_df["INCLUDE_IN_ENTITY"],
                        "DERIVED_EXPRESSION": source_df["DERIVED_EXPRESSION"],
                        "AGGREGATION_FUNCTION": source_df["AGGREGATION_FUNCTION"],
                        "LAST_UPDATED_TIMESTAMP": source_df["LAST_UPDATED_TIMESTAMP"]
                    }
                )
            ]
        )

        return "Operation Successful"
    except:
        return "Operation Failed"
:::
;


/* validation holds SP necessary for performing validations on source data */
create or replace schema validation comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';

create or replace view validation.validation_rule as select * from data_model_mapper_app_package.admin.validation_rule;

create or replace procedure validation.validate(mapped_source_name string)
returns string
language python
runtime_version=3.9
handler='main'
comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
packages=('snowflake-snowpark-python')
AS
:::
import json
import snowflake.snowpark.functions as F
from snowflake.snowpark.exceptions import SnowparkSQLException


def main(session,mapped_source_name):
    
    def get_data_type(d_type: str) -> str:
        """Takes the expected data type from validation rules, and produces the applicable IS_<object> Snowflake type function

        Args:
            d_type: The expected data type of the column to be validated

        Returns:
            The applicable IS_<object> Snowflake type function
        """

        # DECIMAL(10,0) columns are inherintly considered INTEGER columns and will pass both IS_DECIMAL and IS_INTEGER (DECIMAL(10,0)  -->  INTEGER)
        dict_data_types = {
            'VARCHAR': 'IS_VARCHAR',
            'DATE': 'IS_DATE',
            'DECIMAL': 'IS_DECIMAL',
            'INTEGER': 'IS_INTEGER',
            'OBJECT': 'IS_OBJECT',
        }

        try: 
            return dict_data_types[d_type]
        except KeyError:
            return 'UNKNOWN'

    
    def get_sorted_params(valid_obj) -> tuple:
        """Extracts the range values from the validation object, and produces the sorted range values for the query

        Args:
            range_values: The values to be sorted

        Returns:
            Tuple of sorted range values for the query
        """

        # DECIMAL(10,0) columns are inherintly considered INTEGER columns and will pass both IS_DEC
        range_values = valid_obj.get('Conditions', [])

        # Must have at least two elements to have a range
        if len(range_values) < 2: 
            raise ValueError('Range values must have at least two elements')
        
        #Returning the sorted values in proper formatting for the SQL query
        return repr(min(range_values)), repr(max(range_values))


    # Collecting all rules to validate the target table
    rules_table = f'''SELECT * FROM data_model_mapper_app_package.ADMIN.VALIDATION_RULE WHERE TARGET_TABLE = '{mapped_source_name}';'''
    rules = session.sql(rules_table).collect()
    validation_ts = None

    # Case sensitive table name
    case_sensitive_source = f'"{mapped_source_name}"'    

    # Storing the Validation Run Timestamp
    validation_ts_query = f'''SELECT CURRENT_TIMESTAMP();'''
    validation_ts = session.sql(validation_ts_query).collect()[0][0]

    for rule in rules:
        # Grab all attributes of the rule
        rule_name = rule['RULE_NAME']
        validation_type = rule['VALIDATION_TYPE']       
        column_names = json.loads(rule['COLUMN_NAMES'])     
        operator = rule['OPERATOR']
        validation_params = rule['VALIDATION_PARAMS']
        custom_sql_rule = rule['CUSTOM_SQL_CONDITION']
        valid_descrip = rule['VALIDATION_DESCRIP']

        # DOMAIN_OF_VALUES Validation
        if validation_type == 'DOMAIN_OF_VALUES':
            try: 
                # Parsing parameter set values
                valid_parsed = json.loads(validation_params)
                range_values = valid_parsed['Conditions']
                range_set = ', '.join(f"'{value}'" for value in range_values)

                # Validate records based on validation rule and store count
                # Gather the invalid,valid records counts            
                invalid_query = f'''SELECT COUNT(*) AS DOV_INVALID_CT FROM DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source} WHERE NOT {column_names[0]} {operator} ({range_set});'''
                invalid_count = session.sql(invalid_query).collect()[0]['DOV_INVALID_CT']

                # No. of Records that passed the validation
                passed_query = f'''SELECT COUNT(*) AS DOV_VALID_CT FROM DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source} WHERE {column_names[0]} {operator} ({range_set});'''
                passed_count = session.sql(passed_query).collect()[0]['DOV_VALID_CT']
                
                # Generate PASS/FAIL message based on num. of failed records
                validation_result = invalid_count == 0
                validation_message = f"{invalid_count} invalid records found" if not validation_result else "Validation Passed"

            # If the column is absent in the customer table, the validation will log the error message and continue to the next rule
            except SnowparkSQLException as e:
                invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}').count(), 0
                validation_result = False
                
                # Handling missing values in the table
                error_message = str(e).lower()
                if 'invalid identifier' in error_message:
                    validation_message = f"Error: Invalid identifier. Check if column name exists."
                else:
                    validation_message = f"Error: {str(e)}"
            except Exception as e:
                invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}'), 0
                validation_result = False
                validation_message = f"Error: {str(e)}"     

        # LENGTH Validation
        elif validation_type == 'IS_LENGTH':
            try: 
                # Store length criteria
                valid_parsed = json.loads(validation_params)
                len_value = valid_parsed['Conditions']

                # Validate records based on validation rule and store count
                invalid_query = f'''SELECT COUNT(*) AS LEN_INVALID_CT FROM DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source} WHERE NOT LENGTH({column_names[0]}) {operator} {len_value};'''    
                invalid_count = session.sql(invalid_query).collect()[0]['LEN_INVALID_CT']

                # No. of Records that passed the validation
                passed_query = f'''SELECT COUNT(*) AS LEN_VALID_CT FROM DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source} WHERE LENGTH({column_names[0]}) {operator} {len_value};'''   
                passed_count = session.sql(passed_query).collect()[0]['LEN_VALID_CT']            

                # Generate PASS/FAIL message based on num. of failed records
                validation_result = invalid_count == 0
                validation_message = f"{invalid_count} invalid records found" if not validation_result else "Validation Passed"     
    
            # If the column is absent in the customer table, the validation will log the error message and continue to the next rule
            except SnowparkSQLException as e:
                invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}').count(), 0
                validation_result = False
                
                # Handling missing values in the table
                error_message = str(e).lower()
                if 'invalid identifier' in error_message:
                    validation_message = f"Error: Invalid identifier. Check if column name exists."     
                else:
                    validation_message = f"Error: {str(e)}"
            except Exception as e:
                invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}').count(), 0
                validation_result = False
                validation_message = f"Error: {str(e)}"    

        # NULL Validation
        elif validation_type == 'IS_NULL_OR_EMPTY':    
            # IFNULL(COL,'') != '' --> If you'd like to replace NULLs with empty strings. Reference: https://docs.snowflake.com/en/sql-reference/functions/ifnull
            try:
                invalid_query = f'''SELECT COUNT(*) AS NULL_INVALID_CT FROM DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source} WHERE NOT {column_names[0]} {operator} OR {column_names[0]} = '';'''
                invalid_count = session.sql(invalid_query).collect()[0]['NULL_INVALID_CT']

                # No. of Records that passed the validation
                passed_query = f'''SELECT COUNT(*) AS NULL_VALID_CT FROM DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source} WHERE {column_names[0]} {operator};''' 
                passed_count = session.sql(passed_query).collect()[0]['NULL_VALID_CT']               
                
                # Generate PASS/FAIL message based on num. of failed records
                validation_result = invalid_count == 0
                validation_message = f"{invalid_count} invalid records found" if not validation_result else "Validation Passed"                   
                
            # If the column is absent in the customer table, the validation will log the error message and continue to the next rule
            except SnowparkSQLException as e:
                invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}').count(), 0
                validation_result = False
                
                # Handling missing values in the table
                error_message = str(e).lower()
                if 'invalid identifier' in error_message:
                    validation_message = f"Error: Invalid identifier. Check if column name exists." 
                else: 
                    validation_message = f"Error: {str(e)}"                   

        elif validation_type == 'COLUMN_COMPARE': 
            try: 
                # Store param set values
                valid_parsed = json.loads(validation_params)
                range_values = valid_parsed['Conditions']

                # Store param set values
                valid_parsed = json.loads(validation_params)
                range_values = valid_parsed['Conditions']

                # Validate records based on validation rule and store count
                query = f'''SELECT COUNT(*) AS COMP_INVALID_CT FROM DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source} WHERE NOT ({column_names[0]} {operator} TO_DATE({range_values}));'''
                invalid_count = session.sql(query).collect()[0]['COMP_INVALID_CT']

                # No. of Records that passed the validation
                passed_query = f'''SELECT COUNT(*) AS COMP_VALID_CT FROM DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source} WHERE ({column_names[0]} {operator} TO_DATE({range_values}));'''
                passed_count = session.sql(passed_query).collect()[0]['COMP_VALID_CT']   

                # Generate PASS/FAIL message based on num. of failed records
                validation_result = invalid_count == 0
                validation_message = f"{invalid_count} invalid records found" if not validation_result else "Validation Passed"              

            # If the column is absent in the customer table, the validation will log the error message and continue to the next rule
            except SnowparkSQLException as e:
                invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}').count(), 0
                validation_result = False
                
                # Handling missing values in the table
                error_message = str(e).lower()
                if 'invalid identifier' in error_message:
                    validation_message = f"Error: Invalid identifier. Check if column name exists."      

            except Exception as e:
                invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}').count(), 0
                validation_result = False
                validation_message = f"Error: {str(e)}"                           

        # REQD_COLS_POPULATED Validation
        elif validation_type == 'REQD_COLS_POPULATED':
            # Storing target table as a DataFrame
            invalid_df = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}')
            valid_df = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}')

            try: 
                # Collect invalid records -- any records where Source AND NATIVE_ID aren't populated (future state: any additional n columns)
                invalid_condition = (
                    (F.col(column_names[0]).is_null()) | (F.col(column_names[0]) == '') | 
                    (F.col(column_names[1]).is_null()) | (F.col(column_names[1]) == '')
                )

                # No. of Records that passed the validation
                valid_condition = (
                    (F.col(column_names[0]).isNotNull()) & (F.col(column_names[0]) != '') & 
                    (F.col(column_names[1]).isNotNull()) & (F.col(column_names[1]) != '')
                )

                # Counts of invalid and valid records
                invalid_count = invalid_df.filter(invalid_condition).count()
                passed_count = valid_df.filter(valid_condition).count()

                # Generate PASS/FAIL message based on num. of failed records
                validation_result = invalid_count == 0
                validation_message = f"{invalid_count} invalid records found" if not validation_result else "Validation Passed"     


            # If the column is absent in the customer table, the validation will log the error message and continue to the next rule
            except SnowparkSQLException as e:
                invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}').count(), 0
                validation_result = False
                
                # Handling missing values in the table
                error_message = str(e).lower()
                if 'invalid identifier' in error_message:
                    validation_message = "Error: Invalid identifier. Check if column name exists."  
                else:
                    validation_message = f"Error: {str(e)}"
            except IndexError as e:       
                invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}').count(), 0
                validation_result = False 
                validation_message = "Index Error: Missing columns in data set."

        
        # COUNT Validation
        elif validation_type == 'COUNT':
            try: 
                # Validate records based on validation rule and store count
                invalid_query = f'''SELECT COUNT(*) - COUNT(DISTINCT {column_names[0]}) AS CT_INVALID_CT FROM DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source};'''
                invalid_count = session.sql(invalid_query).collect()[0]['CT_INVALID_CT']

                # No. of Records that passed the validation
                passed_query = f'''SELECT COUNT(DISTINCT {column_names[0]}) AS CT_VALID_CT FROM DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source};''' 
                passed_count = session.sql(passed_query).collect()[0]['CT_VALID_CT']   
            
                # Generate PASS/FAIL message based on num. of failed records
                validation_result = invalid_count == 0
                validation_message = f"{invalid_count} invalid duplicate records found" if not validation_result else "Validation Passed"                   

            # If the column is absent in the customer table, the validation will log the error message and continue to the next rule
            except SnowparkSQLException as e:
                invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}').count(), 0
                validation_result = False
                
                # Handling missing values in the table
                error_message = str(e).lower()
                if 'invalid identifier' in error_message:
                    validation_message = f"Error: Invalid identifier. Check if column name exists."  

        # RANGE Validation
        elif validation_type == 'RANGE':    
            try: 
                # Store param set values
                valid_parsed = json.loads(validation_params)

                # Get the min and max values from the range
                # Validate records based on validation rule and store count
                min_value, max_value = get_sorted_params(valid_parsed)
                invalid_query = f'''SELECT COUNT(*) AS RGE_INVALID_CT FROM DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source} WHERE NOT {column_names[0]} {operator} {min_value} AND {max_value};'''
                invalid_count = session.sql(invalid_query).collect()[0]['RGE_INVALID_CT']

                # No. of Records that passed the validation
                passed_query = f'''SELECT COUNT(*) AS RGE_VALID_CT FROM DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source} WHERE {column_names[0]} {operator} {min_value} AND {max_value};''' 
                passed_count = session.sql(passed_query).collect()[0]['RGE_VALID_CT']   

                # Generate PASS/FAIL message based on num. of failed records
                validation_result = invalid_count == 0
                validation_message = f"{invalid_count} invalid records found" if not validation_result else "Validation Passed"                       

            # If the column is absent in the customer table, the validation will log the error message and continue to the next rule
            except SnowparkSQLException as e:
                invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}').count(), 0
                validation_result = False
                
                # Handling missing values in the table
                error_message = str(e).lower()
                if 'invalid identifier' in error_message:
                    validation_message = f"Error: Invalid identifier. Check if column name exists."                
            except Exception as e:
                invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}').count(), 0
                validation_result = False
                validation_message = f"Error: {str(e)}"                          

        # DATA_TYPE Validation
        elif validation_type == 'DATA_TYPE':

            # V1 doesn't include type validation before applying the rules
            # If there is a possiblity of having integers as varchars in the dataset, TRY_CAST won't work here because it provides a false positive
            # Instead, we can use the IS_<object> function to check if the column is of type VARCHAR. NOTE: If dataset is medium to large, this will impact query performance
            # query = f'''SELECT COUNT(*) AS INVALID_CT FROM DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source} WHERE TRY_CAST({column_names} AS {d_type}) IS NULL;'     
            
            try: 
                # Store param set values
                valid_parsed = json.loads(validation_params)
                d_type = valid_parsed['Conditions']               

                # Storing the correct function for the data type
                type_bool_predicate = get_data_type(d_type)    
            
                if type_bool_predicate == 'UNKNOWN':
                    # If the data type is unknown, the validation will log the error message
                    invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}').count(), 0                
                    validation_result = False
                    validation_message = 'Unexpected Error: Unknown Data Type provided.'

                else:
                    invalid_query = f'''SELECT COUNT(*) AS DT_INVALID_CT FROM DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source} WHERE NOT {type_bool_predicate}(TO_VARIANT({column_names[0]}));'''
                    invalid_count = session.sql(invalid_query).collect()[0]['DT_INVALID_CT']

                    # No. of Records that passed the validation
                    passed_query = f'''SELECT COUNT(*) AS DT_VALID_CT FROM DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source} WHERE {type_bool_predicate}(TO_VARIANT({column_names[0]}));''' 
                    passed_count = session.sql(passed_query).collect()[0]['DT_VALID_CT']   

                    # Generate PASS/FAIL message based on num. of failed records
                    validation_result = invalid_count == 0
                    validation_message = f"{invalid_count} invalid records found" if not validation_result else "Validation Passed"   


            # If the column is absent in the customer table, the validation will log the error message and continue to the next rule
            except SnowparkSQLException as e:
                invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}').count(), 0
                validation_result = False
                
                # Handling missing values in the table
                error_message = str(e).lower()
                if 'invalid identifier' in error_message:
                    validation_message = f"Error: Invalid identifier. Check if column name exists."    
            except Exception as e:          
                invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}').count(), 0
                validation_result = False
                validation_message = f"Error: {str(e)}"                        
         
        # REGEX Validation
        elif validation_type == 'REGEX':
            try: 
                # Using Snowpark, we can leverage the library functions to apply the regex pattern to the column
                valid_params_dict = json.loads(validation_params) # converting to dict for Snowpark compatibility
                table_df = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}')
                
                # Storing the regex pattern
                pattern = valid_params_dict.get("Conditions")

                # Collecting invalid records that did not match the regex pattern
                invalid_count = table_df.filter(~F.col(column_names[0]).rlike(pattern)).count()

                # No. of Records that passed the validation
                passed_count = table_df.filter(F.col(column_names[0]).rlike(pattern)).count()

                # Generate PASS/FAIL message based on num. of failed records
                validation_result = invalid_count == 0
                validation_message = f"{invalid_count} invalid records that did not match the pattern {pattern}" if not validation_result else "Validation Passed"      
    

            # If the column is absent in the customer table, the validation will log the error message and continue to the next rule
            except SnowparkSQLException as e:
                invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}').count(), 0
                validation_result = False
                
                # Handling missing values in the table
                error_message = str(e).lower()
                if 'invalid identifier' in error_message:
                    validation_message = f"Error: Invalid identifier. Check if column name exists."  
            except Exception as e:  
                invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}').count(), 0
                validation_result = False
                validation_message = f"Error: {str(e)}"                      

        # CUSTOM_SQL Validation
        elif validation_type == 'CUSTOM_SQL':
            try:
                # Validate records based on validation rule and store count
                invalid_query = f'''SELECT COUNT(*) AS CUSTOM_INVALID_CT FROM DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source} WHERE ({custom_sql_rule});'''
                invalid_count = session.sql(invalid_query).collect()[0]['CUSTOM_INVALID_CT']

                # No. of Records that passed the validation
                passed_query = f'''SELECT COUNT(*) AS CUSTOM_VALID_CT FROM DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source} WHERE NOT ({custom_sql_rule});'''
                passed_count = session.sql(passed_query).collect()[0]['CUSTOM_VALID_CT']   
                
                # Generate PASS/FAIL message based on num. of failed records
                validation_result = invalid_count == 0
                validation_message = f"{invalid_count} invalid records found" if not validation_result else "Validation Passed"                

            # If the custom SQL is not written correctly, the validation will log the error message
            except SnowparkSQLException as e:
                invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}').count(), 0
                validation_result = False
                validation_message = 'Unexpected Error: Custom SQL Validation Failed. Provider to resolve.'
              
            except Exception as e:
                invalid_count, passed_count = session.table(f'DATA_MODEL_MAPPER_SHARE_DB.MAPPED.{case_sensitive_source}').count(), 0
                validation_result = False
                validation_message = f"Error: {str(e)}"

        # Log Table Insertion
        session.sql(f"""INSERT INTO DATA_MODEL_MAPPER_SHARE_DB.VALIDATED.VALIDATION_LOG(RUN_TS,TARGET_TABLE,RULE_NAME,VALIDATION_TYPE,VALIDATION_DESCRIP,COLUMN_NAMES,OPERATOR,VALIDATION_PARAMS,VALIDATION_RESULT,VALIDATION_NUM_PASSED, VALIDATION_NUM_FAILED, VALIDATION_MSG) SELECT 
       '{validation_ts}','{mapped_source_name}','{rule_name}','{validation_type}','{valid_descrip}', parse_json('{json.dumps(column_names)}') , '{operator}', '{validation_params}','{validation_result}', '{passed_count}','{invalid_count}', '{validation_message}';""").collect()

    # Return Output: Validation Timestamp -- to be used as the filter for the latest validation run
    return validation_ts

:::;

create or replace procedure validation.create_validation_task(mapped_source_name string, lag_time string)
returns string
language python
runtime_version=3.9
handler='main'
comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
packages=('snowflake-snowpark-python')
AS
:::
def main(session, mapped_source_name, lag_time):
    session.sql(f'''CREATE TASK IF NOT EXISTS DATA_MODEL_MAPPER_SHARE_DB.VALIDATED."{mapped_source_name}_TASK" WAREHOUSE = data_model_mapper_app_wh SCHEDULE = '{lag_time}' AS CALL data_model_mapper_app.VALIDATION.VALIDATE('{mapped_source_name}');''').collect()
    session.sql(f'''ALTER TASK DATA_MODEL_MAPPER_SHARE_DB.VALIDATED."{mapped_source_name}_TASK" RESUME;''').collect()
    return None
:::;


/* mapping holds the dynamic tables used prior to mapping to target entities */
create or alter versioned schema mapping comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';

/* python procedure to generate dynamic sql */
create or replace procedure modeling.generate_collection_model(source_collection_name varchar)
returns string
language python
runtime_version=3.8
packages=('snowflake-snowpark-python')
handler='generate_collection_model'
comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
as
:::
from snowflake.snowpark.functions import call_udf, col, current_timestamp, when_matched, when_not_matched, lit, \
any_value as any_value_, avg as avg_, corr as corr_, count as count_, covar_pop as covar_pop_, covar_samp as covar_samp_, \
listagg as listagg_, max as max_, median as median_, min as min_, mode as mode_, percentile_cont as percentile_cont_, \
stddev as stddev_, stddev_pop as stddev_pop_, stddev_samp as stddev_samp_, sum as sum_, \
var_pop as var_pop_, var_samp as var_samp_, variance as variance_
from snowflake.snowpark.types import IntegerType, StringType, StructType, StructField
import re

# function to enrich dynamic_df
def enrich_dataframe(session, source_entity_df, base_entity_name, dynamic_df):
    source_collection_name = source_entity_df \
        .select(col("SOURCE_COLLECTION_NAME")).collect()[0][0]
    
    # construct group by
    base_col_list = dynamic_df.columns

    group_dmm_string = ""

    for item in base_col_list:
        if group_dmm_string == "":
            group_dmm_string = 'col("{item}")'.format(item=item)
        else:
            group_dmm_string += ', col("{item}")'.format(item=item)

    col_aggregation_dict = {}

    # recursive CTE to get all related offspring, ordered by depth, then entity name
    offspring_sql = '''
    with recursive offspring
(
        source_collection_name
    ,   source_entity_name
    ,   entity_fully_qualified_source
    ,   is_base_entity
    ,   join_from_source_entity_name
    ,   join_type
    ,   depth
)
as
(
    select
            se.source_collection_name
        ,   se.source_entity_name
        ,   se.entity_fully_qualified_source
        ,   se.is_base_entity
        ,   se.join_from_source_entity_name
        ,   se.join_type
        ,   '*' as depth
    from DATA_MODEL_MAPPER_SHARE_DB.configuration.source_entity se
    where is_base_entity = true and source_collection_name = '{source_collection_name}'

    union all

    select
            se.source_collection_name
        ,   se.source_entity_name
        ,   se.entity_fully_qualified_source
        ,   se.is_base_entity
        ,   se.join_from_source_entity_name
        ,   se.join_type
        ,   o.depth || '*'
    from DATA_MODEL_MAPPER_SHARE_DB.configuration.source_entity se
    inner join offspring o on se.join_from_source_entity_name = o.source_entity_name
)

select
        se.source_collection_name
    ,   se.source_entity_name
    ,   se.entity_fully_qualified_source
    ,   se.is_base_entity
    ,   se.join_from_source_entity_name
    ,   se.join_type
from offspring se
where se.is_base_entity = false
order by depth asc, source_entity_name asc;
    '''.format(source_collection_name=source_collection_name)

    child_entity_list = session.sql(offspring_sql).collect()
    
    # if child entities exist, start joining
    for child_entity_row in child_entity_list:
        join_type = child_entity_row["JOIN_TYPE"]
        
        # get child details
        child_entity_name = child_entity_row["SOURCE_ENTITY_NAME"]
        child_entity_fully_qualified = child_entity_row["ENTITY_FULLY_QUALIFIED_SOURCE"]

        # create dataframe to join
        to_join_df = session.table(child_entity_fully_qualified)

        ########## ATTRIBUTES ##########
        # get columns in to_join_df
        to_join_df = get_attributes(session, source_entity_df, child_entity_name, to_join_df)
        to_join_col_list = to_join_df.columns

        # get list of aggregations
        to_join_agg_list = get_aggregations(session, source_entity_df, child_entity_name, to_join_df)

        # string for column selection and aliasing statement
        column_select_and_alias_string = ""
        
        # get current dynamic_df columns
        current_col_list = dynamic_df.columns

        # add all current dynamic_df cols to the string
        for item in current_col_list:
            if column_select_and_alias_string == "":
                column_select_and_alias_string = 'col("{item}")'.format(item=item)
            else:
                column_select_and_alias_string += ', col("{item}")'.format(item=item)

        # find matching columns
        matched_col_list = list(set(current_col_list).intersection(to_join_col_list))

        # string for column preparation
        to_join_column_prep_string = ""
        
        # build up col select/alias statement
        for idx, item in enumerate(to_join_col_list):
            matching_item_found = False   

            for matched_item in matched_col_list:
                if matched_item == item:
                    matching_item_found = True

                    # alias duplicate cols by prepending <child_entity_name>__
                    if to_join_column_prep_string == "":
                        to_join_column_prep_string = 'col("{item}").alias("{child_entity_name}__{item}")'.format(child_entity_name=child_entity_name,item=item)
                    else:
                        to_join_column_prep_string += ', col("{item}").alias("{child_entity_name}__{item}")'.format(child_entity_name=child_entity_name,item=item)

                    if to_join_agg_list[idx]:
                        col_aggregation_dict[child_entity_name.upper() + "__" + item.upper()] = to_join_agg_list[idx]
            
            # add to string if not a duplicate
            if not matching_item_found:
                if to_join_column_prep_string == "":
                    to_join_column_prep_string = 'col("{item}")'.format(item=item)
                else:
                    to_join_column_prep_string += ', col("{item}")'.format(item=item)

                if to_join_agg_list[idx]:
                        col_aggregation_dict[item.upper()] = to_join_agg_list[idx]

        to_join_column_prep_statement = "to_join_df.select({to_join_column_prep_string})".format(to_join_column_prep_string=to_join_column_prep_string)

        # updates column names to prevent duplicates
        to_join_df = eval(to_join_column_prep_statement)

        ########## JOIN CONDITIONS ##########
        # get join conditions
        join_conditions_list = session.table("DATA_MODEL_MAPPER_SHARE_DB.configuration.source_entity_join_condition") \
            .filter((col("SOURCE_COLLECTION_NAME") == source_collection_name) & \
                (col("SOURCE_ENTITY_NAME") == child_entity_name) & \
                (col("JOIN_FROM_SOURCE_ENTITY_NAME") == base_entity_name)).collect()

        # string for creating join condition statement
        join_condition_string = ""

        # iterate through join conditions
        for join_condition_row in join_conditions_list:
            join_from_attribute_name = join_condition_row["JOIN_FROM_ENTITY_ATTRIBUTE_NAME"].upper()
            sql_operator = join_condition_row["OPERATOR"]
            join_to_attribute_name = join_condition_row["JOIN_TO_ENTITY_ATTRIBUTE_NAME"].upper()

            # check if join_to attribute is a duplicate, if so, update to prepend <child_entity_name>__
            for matched_item in matched_col_list:
                if join_to_attribute_name == matched_item:
                    join_to_attribute_name = child_entity_name + "__" + join_to_attribute_name

            # convert sql comparison operator to python equivalent
            if sql_operator == "=":
                sp_operator = "=="
            elif sql_operator == ">=":
                sp_operator = ">="
            elif sql_operator == "<=":
                sp_operator = "<="
            elif sql_operator == ">":
                sp_operator = ">"
            elif sql_operator == "<":
                sp_operator = "<"
            elif sql_operator == "<>":
                sp_operator = "!="
            else:
                return "Invalid operator found"

            # assumes & operator only
            if join_condition_string == "":
                join_condition_string = "(dynamic_df.{join_from_attribute_name} {sp_operator} to_join_df.{join_to_attribute_name})" \
                    .format(join_from_attribute_name=str(join_from_attribute_name), sp_operator=sp_operator, join_to_attribute_name=str(join_to_attribute_name))
            else:
                join_condition_string += " & (dynamic_df.{join_from_attribute_name} {sp_operator} to_join_df.{join_to_attribute_name})" \
                    .format(join_from_attribute_name=str(join_from_attribute_name), sp_operator=sp_operator, join_to_attribute_name=str(join_to_attribute_name))

        # join dynamic_df and new to_join_df based on one-to-many join conditions
        join_statement = "dynamic_df.join(to_join_df, {join_condition_string})".format(join_condition_string=str(join_condition_string))

        # updates dynamic_df by running join statement
        dynamic_df = eval(join_statement)

    column_list = dynamic_df.columns

    # create group by expr statement and eval
    group_dmm_exp_statement = "dynamic_df.groupBy({group_dmm_string})".format(group_dmm_string=group_dmm_string)
    dynamic_df = eval(group_dmm_exp_statement)

    aggregation_string = ""

    # build aggregation string
    for column in column_list:
        if column in col_aggregation_dict:
            if aggregation_string == "":
                aggregation_string = '{aggregation}_("{column}").alias("{column}")'.format(column=column,aggregation=col_aggregation_dict[column].lower())
            else:
                aggregation_string += ', {aggregation}_("{column}").alias("{column}")'.format(column=column,aggregation=col_aggregation_dict[column].lower())

    agg_exp_statement = "dynamic_df.agg({aggregation_string})".format(aggregation_string=aggregation_string)
    dynamic_df = eval(agg_exp_statement)

    return dynamic_df


# function to get aggregations
def get_aggregations(session, source_entity_df, source_entity_name, source_df): 
    # get source collection name
    source_collection_name = source_entity_df \
        .select(col("SOURCE_COLLECTION_NAME")).collect()[0][0]

    col_list = source_df.columns

    aggregation_list = []

    for col_name in col_list:
        # remove source entity name from a detected duplicate
        col_name = re.sub(source_entity_name + "__", "", col_name, 1)

        # get attribute
        aggregation_function = session.table('DATA_MODEL_MAPPER_SHARE_DB.configuration.source_entity_attribute') \
            .filter( \
                (col("SOURCE_COLLECTION_NAME") == source_collection_name) &  \
                (col("SOURCE_ENTITY_NAME") == source_entity_name) &  \
                (col("SOURCE_ENTITY_ATTRIBUTE_NAME") == col_name)) \
            .select(col("AGGREGATION_FUNCTION")).collect()[0][0]

        aggregation_list.append(aggregation_function)

    return aggregation_list


# function to get attributes
def get_attributes(session, source_entity_df, source_entity_name, source_df):
    # store dataframe we will dynamically update
    altered_source_df = source_df
    
    # get source collection name
    source_collection_name = source_entity_df \
        .select(col("SOURCE_COLLECTION_NAME")).collect()[0][0]

    # get attributes
    all_attributes_df = session.table('DATA_MODEL_MAPPER_SHARE_DB.configuration.source_entity_attribute') \
        .filter( \
            (col("SOURCE_COLLECTION_NAME") == source_collection_name) &  \
            (col("SOURCE_ENTITY_NAME") == source_entity_name))

    # if no attributes, generate them
    if all_attributes_df.count() == 0:
        procedure_call_statement = "call modeling.generate_attributes('{source_collection_name}', '{source_entity_name}')".format(source_collection_name=source_collection_name, source_entity_name = source_entity_name)
        session.sql(procedure_call_statement).collect()

        all_attributes_df = session.table('DATA_MODEL_MAPPER_SHARE_DB.configuration.source_entity_attribute') \
            .filter( \
                (col("SOURCE_COLLECTION_NAME") == source_collection_name) &  \
                (col("SOURCE_ENTITY_NAME") == source_entity_name))

    # get attribute df, filtering by collection, entity, and if included
    attributes_df = all_attributes_df.filter(col("INCLUDE_IN_ENTITY") == True)

    # get non-derived attribute names
    non_derived_attribute_names_df = attributes_df.filter(col("DERIVED_EXPRESSION").isNull()) \
        .select(col("SOURCE_ENTITY_ATTRIBUTE_NAME"))

    # get derived attribute names
    derived_attribute_names_df = attributes_df.filter(col("DERIVED_EXPRESSION").isNotNull()) \
        .select(col("SOURCE_ENTITY_ATTRIBUTE_NAME"), col("DERIVED_EXPRESSION"))

    # string to select included columns, will eventually use select_expr
    col_select_string = ""

    # get attributes
    for row in non_derived_attribute_names_df.to_local_iterator():
        if col_select_string == "":
            col_select_string += '"{col}"'.format(col=row[0])
        else:
            col_select_string += ', "{col}"'.format(col=row[0])

    # index 1 is the derived expression, we'll update the col name later
    for row in derived_attribute_names_df.to_local_iterator():
        if col_select_string == "":
            col_select_string += '"{expr}"'.format(expr=row[1]) 
        else:
            col_select_string += ', "{expr}"'.format(expr=row[1])

    # create select expr statement and eval
    select_exp_statement = "altered_source_df.select_expr({col_select_string})".format(col_select_string=col_select_string)
    altered_source_df = eval(select_exp_statement)

    # update col name for derived cols
    for row in derived_attribute_names_df.to_local_iterator():
        with_col_rename_statement = 'altered_source_df.with_column_renamed("{existing_col}", "{new_col}")'.format(existing_col=row[1].upper(), new_col=row[0])
        altered_source_df = eval(with_col_rename_statement)

    return altered_source_df


# filter dataframe
def filter_dataframe(session, source_collection_name, dynamic_df):
    filter_conditions_df = session.table('DATA_MODEL_MAPPER_SHARE_DB.configuration.source_collection_filter_condition') \
        .filter(col("SOURCE_COLLECTION_NAME") == source_collection_name)

    # return same dataframe if no filter conditions
    if filter_conditions_df.count() == 0:
        return dynamic_df
    else:
        filter_statement = ""
        
        for row in filter_conditions_df.to_local_iterator():
            left_expr = row["LEFT_FILTER_EXPRESSION"]
            sql_operator = row["OPERATOR"]
            right_expr = row["RIGHT_FILTER_EXPRESSION"]
            
            # only support ANDs to prevent overly complicating UI
            if filter_statement == "":
                filter_statement = '{left_expr} {sql_operator} {right_expr}'.format(left_expr=left_expr,sql_operator=sql_operator,right_expr=right_expr)
            else:
                filter_statement += 'AND {left_expr} {sql_operator} {right_expr}'.format(left_expr=left_expr,sql_operator=sql_operator,right_expr=right_expr)
        
        # create filter expr statement and eval
        filter_exp_statement = 'dynamic_df.filter("{filter_statement}")'.format(filter_statement=filter_statement)
        dynamic_df = eval(filter_exp_statement)

        return dynamic_df


# entry point function
def generate_collection_model(session, source_collection_name):
    # get source entity df for collection
    source_entity_df = session.table('DATA_MODEL_MAPPER_SHARE_DB.configuration.source_entity') \
        .filter(col("SOURCE_COLLECTION_NAME") == source_collection_name)

    # check if custom sql
    use_custom_sql = session.table('DATA_MODEL_MAPPER_SHARE_DB.configuration.source_collection') \
        .filter(col("SOURCE_COLLECTION_NAME") == source_collection_name) \
        .select(col("USE_CUSTOM_SQL")) \
        .collect()[0][0]

    if use_custom_sql:
        custom_sql_string = session.table('DATA_MODEL_MAPPER_SHARE_DB.configuration.source_collection') \
            .filter(col("SOURCE_COLLECTION_NAME") == source_collection_name) \
            .select(col("CUSTOM_SQL")) \
            .collect()[0][0]

        dynamic_df = session.sql(custom_sql_string)
    else:
        # get df with no join froms (should only be one per collection)
        base_entity_df = source_entity_df \
            .filter(col("IS_BASE_ENTITY") == True)

        # get the name for the source entity
        base_entity_name = str(base_entity_df \
            .select(col("SOURCE_ENTITY_NAME")) \
            .collect()[0][0])

        # get the fully qualified source for the source entity
        base_entity_fully_qualified = str(base_entity_df \
            .select(col("ENTITY_FULLY_QUALIFIED_SOURCE")) \
            .collect()[0][0])

        # update dynamic df - we will use this to collect our transformations as we go
        dynamic_df = session.table(base_entity_fully_qualified)

        # get attributes
        dynamic_df = get_attributes(session, source_entity_df, base_entity_name, dynamic_df)

        # method to add relevant, recursive joins and transformations to dynamic_df
        dynamic_df = enrich_dataframe(session, source_entity_df, base_entity_name, dynamic_df)

        # add filtering
        dynamic_df = filter_dataframe(session, source_collection_name, dynamic_df)

    # ensure that the collection name meets normal table naming rules
    generated_mapping_table = "DATA_MODEL_MAPPER_SHARE_DB.modeled." + source_collection_name.replace(" ","_")
 
    # save table name to source collection
    session.sql("""
        update DATA_MODEL_MAPPER_SHARE_DB.configuration.source_collection
        set generated_mapping_table = '{generated_mapping_table}'
        where source_collection_name = '{source_collection_name}'
    """.format(generated_mapping_table=generated_mapping_table, source_collection_name=source_collection_name)).collect()

    # create a dynamic table in mapping schema
    dynamic_df.create_or_replace_dynamic_table(name=generated_mapping_table, warehouse="data_model_mapper_app_wh", lag="DOWNSTREAM")

    session.sql(f"""ALTER TABLE {generated_mapping_table} set comment= '\u007b"origin":"sf_sit","name":"dmm","version":\u007b"major":1, "minor":0\u007d,"attributes":
    \u007b"component":"dmm"\u007d\u007d'""").collect()

    session.sql("""
        grant select on table {generated_mapping_table} to application role dmm_consumer_app_role with grant option
    """.format(generated_mapping_table=generated_mapping_table)).collect()

    return "Collection dynamic table created"
:::
;

/* python procedure get columns from the source collection dynamic tables to map-from, used by Streamlit to grab columns */
create or replace procedure mapping.get_mapfrom_columns(source_collection_name varchar)
returns string
language python
runtime_version=3.8
packages=('snowflake-snowpark-python')
handler='get_mapfrom_columns'
comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
as
:::
from snowflake.snowpark.functions import call_udf, col, current_timestamp, when_matched, when_not_matched
from snowflake.snowpark.types import IntegerType, StringType, StructType, StructField

def get_mapfrom_columns(session, source_collection_name):
    # source collection
    source_collection_df = session.table('DATA_MODEL_MAPPER_SHARE_DB.configuration.source_collection') \
        .filter(col('source_collection_name') == source_collection_name)

    # generated table
    generated_mapping_table = source_collection_df.select(col('generated_mapping_table')).collect()[0][0]

    # table df
    generated_mapping_table_df = session.table(generated_mapping_table)

    return generated_mapping_table_df.columns
:::
;

/* python procedure get attributes for target entities to map-to, used by Streamlit to grab columns */
create or replace procedure mapping.get_mapto_columns(source_collection_name varchar)
returns string
language python
runtime_version=3.8
packages=('snowflake-snowpark-python')
handler='get_mapto_columns'
comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
as
:::
from snowflake.snowpark.functions import call_udf, col, current_timestamp, when_matched, when_not_matched
from snowflake.snowpark.types import IntegerType, StringType, StructType, StructField

def get_mapto_columns(session, source_collection_name):
    # source collection
    source_collection_df = session.table('DATA_MODEL_MAPPER_SHARE_DB.configuration.source_collection') \
        .filter(col('source_collection_name') == source_collection_name)

    # target collection
    target_collection_name = source_collection_df.select(col('target_collection_name')).collect()[0][0]

    # custom SQL

    # version
    version = source_collection_df.select(col('version')).collect()[0][0]

    # target entity
    target_entity_name = source_collection_df.select(col('target_entity_name')).collect()[0][0]

    # target entity attribute df
    target_entity_attribute_df = session.table('admin.target_entity_attribute') \
        .filter( \
            (col('target_collection_name') == target_collection_name) & \
            (col('version') == version) & \
            (col('target_entity_name') == target_entity_name))

    return target_entity_attribute_df.columns
:::
;

/* python procedure to generate the secure view in the share database */
create or replace procedure mapping.generate_view(source_collection_name varchar)
returns string
language python
runtime_version=3.8
packages=('snowflake-snowpark-python')
handler='generate_view'
comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
as
:::
from snowflake.snowpark.functions import call_udf, col, current_timestamp, when_matched, when_not_matched
from snowflake.snowpark.types import IntegerType, StringType, StructType, StructField

def generate_view(session, source_collection_name):
    # get current database (app name)
    application_name = session.sql("""
        select current_database()
    """).collect()[0][0]
    
    # source collection
    source_collection_df = session.table('DATA_MODEL_MAPPER_SHARE_DB.configuration.source_collection'.format(application_name=application_name)) \
        .filter(col('source_collection_name') == source_collection_name)

    # generated table
    generated_mapping_table = source_collection_df.select(col('generated_mapping_table')).collect()[0][0]

    dynamic_df = session.table(generated_mapping_table)

    # source to target mapping
    source_to_target_mapping_df = session.table('DATA_MODEL_MAPPER_SHARE_DB.configuration.source_to_target_mapping'.format(application_name=application_name)) \
        .filter(col('source_collection_name') == source_collection_name)

    # string to select included columns, will eventually use select_expr
    col_select_string = ""

    # get attributes
    for row in source_to_target_mapping_df.to_local_iterator():
        if col_select_string == "":
            col_select_string += '"{col}"'.format(col=row[1])
        else:
            col_select_string += ', "{col}"'.format(col=row[1])

    # create select expr statement and eval
    select_exp_statement = "dynamic_df.select_expr({col_select_string})".format(col_select_string=col_select_string)
    dynamic_df = eval(select_exp_statement)

    # update col name for derived cols
    for row in source_to_target_mapping_df.to_local_iterator():
        with_col_rename_statement = """dynamic_df.with_column_renamed("{existing_col}", "{new_col}")""".format(existing_col=row[1].upper(), new_col=row[2])
        dynamic_df = eval(with_col_rename_statement)

    # target collection
    target_collection_name = source_collection_df.select(col('target_collection_name')).collect()[0][0]

    # version
    version = source_collection_df.select(col('version')).collect()[0][0]

    # target entity
    target_entity_name = source_collection_df.select(col('target_entity_name')).collect()[0][0]

    view_name = '"DATA_MODEL_MAPPER_SHARE_DB"."MAPPED"."' + target_collection_name + '__' + version + '__' + target_entity_name + '"'
    
    dynamic_df.create_or_replace_view(view_name)

    session.sql(f"""ALTER VIEW {view_name} set comment= '\u007b"origin":"sf_sit","name":"dmm","version":\u007b"major":1, "minor":0\u007d,"attributes":
    \u007b"component":"dmm"\u007d\u007d'""").collect()

    session.sql('alter view ' + view_name + ' set secure').collect()

    session.sql("""
        grant select on view {view_name} to application role dmm_consumer_app_role with grant option
    """.format(view_name=view_name)).collect()

    return 'Secure view generated successfully'
:::
;

/* create user_interface schema */
create or alter versioned schema user_interface;

/* create reference procedure */
/* this callback is used by the UI to ultimately bind a reference that expects one value */
create or replace procedure user_interface.register_single_callback(ref_name string, operation string, ref_or_alias string)
returns string
language sql
as :::
    begin
        case (operation)
            when 'ADD' then
                select system$set_reference(:ref_name, :ref_or_alias);
            when 'REMOVE' then
                select system$remove_reference(:ref_name);
            when 'CLEAR' then
                select system$remove_reference(:ref_name);
            else
                return 'Unknown operation: ' || operation;
        end case;
        return 'Operation ' || operation || ' succeeded';
    end;
:::
;

/* create Streamlits */
create or replace streamlit user_interface.data_modeler_streamlit
from '/code_artifacts/streamlit'
main_file = '/data_modeler.py'
;

grant usage on schema user_interface to application role dmm_consumer_app_role;
grant usage on streamlit user_interface.data_modeler_streamlit to application role dmm_consumer_app_role;

grant usage on schema modeling to application role dmm_consumer_app_role;
grant usage on procedure modeling.initialize_application() to application role dmm_consumer_app_role;

grant usage on schema modeling to application role dmm_consumer_app_role;
grant select on all tables in schema modeling to application role dmm_consumer_app_role;
grant usage on procedure modeling.generate_attributes(varchar, varchar) to application role dmm_consumer_app_role;
grant usage on procedure modeling.generate_collection_model(varchar) to application role dmm_consumer_app_role;

grant usage on schema validation to application role dmm_consumer_app_role;
grant usage on procedure validation.validate(varchar) to application role dmm_consumer_app_role;
grant usage on procedure validation.create_validation_task(varchar, varchar) to application role dmm_consumer_app_role;
grant select on all tables in schema validation to application role dmm_consumer_app_role;
grant select on view validation.validation_rule to application role dmm_consumer_app_role;

grant usage on schema mapping to application role dmm_consumer_app_role;
grant select on all tables in schema mapping to application role dmm_consumer_app_role;
grant usage on procedure mapping.get_mapfrom_columns(varchar) to application role dmm_consumer_app_role;
grant usage on procedure mapping.get_mapto_columns(varchar) to application role dmm_consumer_app_role;
grant usage on procedure mapping.generate_view(varchar) to application role dmm_consumer_app_role;

/* create application role for demo data access - not for normal usage */
create or replace application role dmm_demo_app_role;

$$,':::','$$') as CONTENT;

In [ ]:
insert into data_model_mapper_deployment_db.code.file (NAME , CONTENT)
values ( 'README',$$
# Data Mappper
      
The Data Mappper faciliates modeling, mapping, and sharing consumer data with the provider in a standard format.

$$);

In [ ]:
/* add environment file for streamlit - includes all references to libraries that the Streamlit needs */
insert into data_model_mapper_deployment_db.code.file (name , content)
values ( 'ENVIRONMENT',$$
name: sf_env
channels:
- snowflake
dependencies:
- snowflake-native-apps-permission
- streamlit=1.39.0
- editdistance

$$);

In [ ]:
/* add the Streamlit file */
insert into data_model_mapper_deployment_db.code.file (name , content)
values ( 'STREAMLIT',$$
from appPages.page import st
from appPages.collection_list import CollectionList
from appPages.collection_joining import CollectionJoining
from appPages.collection_mapping import CollectionMapping
from appPages.entity_configure import EntityConfiguration
from appPages.validation import ValidationPage
from appPages.validations_history import ValidationsHistoryPage
from appPages.initial_setup import InitialSetupPage
from snowflake.snowpark.context import get_active_session


# Check snowflake connection type

def set_session():
    try:
        import snowflake.permissions as permissions

        session = get_active_session()

        st.session_state["streamlit_mode"] = "NativeApp"
    except:
        try:
            session = get_active_session()

            st.session_state["streamlit_mode"] = "SiS"
        except:
            import snowflake_conn as sfc

            session = sfc.init_snowpark_session("account_1")

            st.session_state["streamlit_mode"] = "OSS"

    return session


# Set user organization name
# if "organization_name" not in st.session_state:
#     st.session_state.organization_name = session.sql(
#         "SELECT CURRENT_ORGANIZATION_NAME()"
#     ).collect()[0]


# Set starting page
if "page" not in st.session_state:
    st.session_state.page = "collection_list"

    #Set table database location session variables
    #This should happen on first load only
    st.session_state.native_database_name = "data_model_mapper_share_db"

pages = [
    CollectionList(),
    CollectionJoining(),
    CollectionMapping(),
    EntityConfiguration(),
    ValidationPage(),
    ValidationsHistoryPage(),
    InitialSetupPage()
]


def main():
    st.set_page_config(layout="wide")
    if 'session' not in st.session_state:
        st.session_state.session = set_session()
    for page in pages:
        if page.name == st.session_state.page:
            st.session_state.layout = "wide"
            page.print_sidebar()
            page.print_page()


main()

$$);

And add a stage for our version/patch with the files

In [ ]:

create or replace stage data_model_mapper_deployment_db.code.files_v1_p0 comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';

/* put files into stage */
CALL data_model_mapper_deployment_db.code.PUT_TO_STAGE('files_v1_p0','manifest.yml',(SELECT CONTENT FROM data_model_mapper_deployment_db.code.file WHERE NAME = 'MANIFEST'));
CALL data_model_mapper_deployment_db.code.PUT_TO_STAGE('files_v1_p0','setup_script.sql', (SELECT CONTENT FROM data_model_mapper_deployment_db.code.file WHERE NAME = 'SETUP'));
CALL data_model_mapper_deployment_db.code.PUT_TO_STAGE('files_v1_p0','README.md', (SELECT CONTENT FROM data_model_mapper_deployment_db.code.file WHERE NAME = 'README'));
CALL data_model_mapper_deployment_db.code.PUT_TO_STAGE('files_v1_p0/code_artifacts/streamlit','environment.yml', (SELECT CONTENT FROM data_model_mapper_deployment_db.code.file WHERE NAME = 'ENVIRONMENT'));
CALL data_model_mapper_deployment_db.code.PUT_TO_STAGE('files_v1_p0/code_artifacts/streamlit','data_modeler.py', (SELECT CONTENT FROM data_model_mapper_deployment_db.code.file WHERE NAME = 'STREAMLIT'));

Stage pickle of Streamlit files and images for app.

Note, if you would like to confirm the pickle string is the same, please check out the file_pickler notebook and run it locally with the files.  The resulting string should match.

In [ ]:


from snowflake.snowpark.context import get_active_session
session = get_active_session()
session.sql('use role accountadmin;').collect()

create_tbl_sql = """create or replace table dmm_test_db.code.file (
        NAME varchar
    ,   CONTENT varchar(16777216)
) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';"""
session.sql(create_tbl_sql).collect()

In [ ]:
insert into dmm_test_db.code.file (NAME, CONTENT)
values ('PICKLE', '...
')


In [ ]:
import pickle
import codecs
import os
import base64

from snowflake.snowpark.context import get_active_session
session = get_active_session()
session.sql('use role data_model_mapper_role;').collect()

def decode(obj):
    pickled = codecs.decode(obj.encode(), 'base64')
    return pickle.loads(pickled)

encoded_obj = session.sql("select content from dmm_test_db.code.file where NAME = 'PICKLE'").collect()[0][0]

session.sql('use role data_model_mapper_role;').collect()

# Decode object to confirm it is the same as the original object
decoded_obj = decode(encoded_obj)

# Stage each file
for file_name, file_content in decoded_obj['files'].items():
    # replace double dollars to help with SQL compatibility
    file_content = str(file_content.decode("utf-8")).replace("$$", '"""')
    sql = """call data_model_mapper_deployment_db.code.put_to_stage('files_v1_p0/code_artifacts/streamlit/appPages','{file_name}', $${file_content}$$)""".format(file_name=file_name, file_content=file_content)
    session.sql(sql).collect()

# Stage each image
for image_name, image_content in decoded_obj['images'].items():
    image_content = str(image_content.decode("utf-8"))
    sql = """call data_model_mapper_deployment_db.code.put_image_to_stage('files_v1_p0/code_artifacts/streamlit/Images','{image_name}', $${image_content}$$)""".format(image_name=image_name, image_content=image_content)
    session.sql(sql).collect()


SnowparkSessionException: (1403): No default Session is found. Please create a session before you call function 'udf' or use decorator '@udf'.

Let's prepare the sample data - You can skip this if you are setting up your own data

In [ ]:
/* database and schema for target collection metadata - data that is used to create target collections */
create or replace database dmm_collection_metadata_db comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';
create or replace schema product comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';

create or replace TABLE dmm_collection_metadata_db.product.SITE (SITE_NAME VARCHAR(80), SITE_ID VARCHAR(80), SITE_CITY VARCHAR(16777216),  SITE_STATE VARCHAR(50)) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';
create or replace TABLE dmm_collection_metadata_db.product.PRODUCT (PRODUCT_NAME VARCHAR(200), PRODUCT_ID NUMBER(38,0), COST_PER_ITEM FLOAT, PRODUCT_DESCRIPTION VARCHAR(16777216), PRODUCT_PRICE FLOAT, PRODUCT_COST FLOAT) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';
create or replace TABLE dmm_collection_metadata_db.product.INVENTORY (SITE_ID VARCHAR(80), PRODUCT_ID NUMBER(38,0), AMOUNT FLOAT, LAST_COUNTED_DATE TIMESTAMP_NTZ(9)) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';

create or replace schema mock_data comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';
create or replace TABLE dmm_collection_metadata_db.mock_data.CUSTOMER (SOURCE VARCHAR(80), NATIVE_ID VARCHAR(80), NAME_PREFIX VARCHAR(50), FIRST_NAME VARCHAR(50), MIDDLE_NAME VARCHAR(50), LAST_NAME VARCHAR(50), MAIDEN_NAME VARCHAR(50), PERSON_SUFFIX VARCHAR(50), GENDER CHAR(1), FAKE_SSN VARCHAR(50), DOB DATE, DECEASED CHAR(1)) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';
create or replace TABLE dmm_collection_metadata_db.mock_data.CUSTOMER_INFO (NATIVE_ID VARCHAR(80), PHONE_AREA_CODE VARCHAR(50), PHONE_NUMBER VARCHAR(50), EMAIL VARCHAR(50), STREET_ADDRESS VARCHAR(50), CITY VARCHAR(50), STATE VARCHAR(50), ZIPCODE VARCHAR(50)) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';

--------------------------------------------------------------------------------------------------------------------------------------------------
-- Load target collection metadata to the attribute table
-- Stored procedure to update the entity tables
---------------------------------------------------------------------------------------------------------------------------------------------------
create or replace procedure dmm_collection_metadata_db.product.generate_attributes(attr_table varchar, target_collection_name varchar, target_entity_name varchar, version varchar)
returns string
language python
runtime_version=3.8
packages=('snowflake-snowpark-python')
handler='generate_attributes'
comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
as
$$
import snowflake.snowpark
from snowflake.snowpark.functions import call_udf, col, current_timestamp, when_matched, when_not_matched
from snowflake.snowpark.types import IntegerType, StringType, StructType, StructField

def generate_attributes(session:snowflake.snowpark.Session, attr_table:str, target_collection_name:str, target_entity_name:str, version:str):
    try:

        # generates show columns statement for source table
        show_col_sql_text = f"show columns in table {attr_table}"

        session.sql(show_col_sql_text).collect()

        # use the last_query_id to get results of the show col statement
        last_query_results_sql_text = f"""
            select
                    '{target_collection_name}' as target_collection_name
                ,   '{target_entity_name}' as target_entity_name
                ,   "column_name" as target_entity_attribute_name
                ,   object_construct(
                        'data_type',parse_json("data_type"):type::varchar,
                        'is_nullable',parse_json("data_type"):nullable::boolean,
                        'precision',parse_json("data_type"):precision::number,
                        'scale',parse_json("data_type"):scale::number,
                        'length',parse_json("data_type"):length::number,
                        'byte_length',parse_json("data_type"):byteLength::number,
                        'description',null
                    ) as target_attribute_properties

            from table(RESULT_SCAN(LAST_QUERY_ID()))
        """

        source_df = session.sql(last_query_results_sql_text).with_column("LAST_UPDATED_TIMESTAMP", current_timestamp())

        target_df = session.table('data_model_mapper_app_package.ADMIN.TARGET_ENTITY_ATTRIBUTE')
        target_df_collection = session.table('data_model_mapper_app_package.ADMIN.TARGET_COLLECTION')
        target_df_entity = session.table('data_model_mapper_app_package.ADMIN.TARGET_ENTITY')

        # merge with table on pk cols
        target_df.merge(
            source_df,
            (
                (target_df["TARGET_COLLECTION_NAME"] == source_df["TARGET_COLLECTION_NAME"]) &
                (target_df["TARGET_ENTITY_NAME"] == source_df["TARGET_ENTITY_NAME"]) &
                (target_df["TARGET_ENTITY_ATTRIBUTE_NAME"] == source_df["TARGET_ENTITY_ATTRIBUTE_NAME"])
            )
            ,
            [
                when_matched().update(
                    {
                        "TARGET_ATTRIBUTE_PROPERTIES": source_df["TARGET_ATTRIBUTE_PROPERTIES"],
                        "LAST_UPDATED_TIMESTAMP": source_df["LAST_UPDATED_TIMESTAMP"],
                        "VERSION": version           }
                ),
                when_not_matched().insert(
                    {
                        "TARGET_COLLECTION_NAME": source_df["TARGET_COLLECTION_NAME"],
                        "TARGET_ENTITY_NAME": source_df["TARGET_ENTITY_NAME"],
                        "TARGET_ENTITY_ATTRIBUTE_NAME": source_df["TARGET_ENTITY_ATTRIBUTE_NAME"],
                        "TARGET_ATTRIBUTE_PROPERTIES": source_df["TARGET_ATTRIBUTE_PROPERTIES"],
                        "VERSION": version
                    }
                )
            ]
        )

        source_df_text = f"""
            select
                    '{target_collection_name}' as target_collection_name
                ,   '{target_entity_name}' as target_entity_name
        """
        source_df_collect = session.sql(source_df_text).with_column("LAST_UPDATED_TIMESTAMP", current_timestamp())

        target_df_entity.merge(
            source_df_collect,
            (
                (target_df_entity["TARGET_COLLECTION_NAME"] == source_df_collect["TARGET_COLLECTION_NAME"]) &
                (target_df_entity["TARGET_ENTITY_NAME"] == source_df_collect["TARGET_ENTITY_NAME"])
            )
            ,
            [
                when_matched().update(
                    {
                        "LAST_UPDATED_TIMESTAMP": source_df_collect["LAST_UPDATED_TIMESTAMP"],
                        "VERSION": version             }
                ),
                when_not_matched().insert(
                    {
                        "TARGET_COLLECTION_NAME": source_df_collect["TARGET_COLLECTION_NAME"],
                        "TARGET_ENTITY_NAME": source_df_collect["TARGET_ENTITY_NAME"],
                        "VERSION": version
                    }
                )
            ]
        )

        target_df_collection.merge(
            source_df_collect,
            (
                (target_df_entity["TARGET_COLLECTION_NAME"] == source_df_collect["TARGET_COLLECTION_NAME"])
            )
            ,
            [
                when_matched().update(
                    {
                        "LAST_UPDATED_TIMESTAMP": source_df_collect["LAST_UPDATED_TIMESTAMP"],
                        "VERSION": version             }
                ),
                when_not_matched().insert(
                    {
                        "TARGET_COLLECTION_NAME": source_df_collect["TARGET_COLLECTION_NAME"],
                        "VERSION": version
                    }
                )
            ]
        )

        return "Operation Successful"
    except:
        return "Operation Failed"
$$
;

-- Load the metadata from tables to target collection tables
insert into data_model_mapper_app_package.admin.customer(customer_name, customer_snowflake_organization_name) values ('CustomerABC','CustomerABCOrgName');
call dmm_collection_metadata_db.product.generate_attributes('dmm_collection_metadata_db.product.SITE','Product','SITE', 'v1');
call dmm_collection_metadata_db.product.generate_attributes('dmm_collection_metadata_db.product.PRODUCT','Product','PRODUCT', 'v1');
call dmm_collection_metadata_db.product.generate_attributes('dmm_collection_metadata_db.product.INVENTORY','Product','INVENTORY', 'v1');

call dmm_collection_metadata_db.product.generate_attributes('dmm_collection_metadata_db.mock_data.CUSTOMER','Mock_Data','CUSTOMER', 'v1');
call dmm_collection_metadata_db.product.generate_attributes('dmm_collection_metadata_db.mock_data.CUSTOMER_INFO','Mock_Data','CUSTOMER_INFO', 'v1');

insert into data_model_mapper_app_package.admin.subscription(customer_name, target_collection_name, version, expiration_date) values ('CustomerABC','Product','v1','2026-01-01');


/* create sample customer data - this will be the raw data we map to our targets */
create or replace database dmm_customer_sample_db comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';
create or replace schema dmm_customer_sample_db.sample_data comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';
drop schema if exists dmm_customer_sample_db.public;

create or replace TABLE dmm_customer_sample_db.sample_data.LOCATIONS (LOCATION_NAME VARCHAR(80), LOCATION_ID VARCHAR(80), LOCATION_CITY VARCHAR(16777216),  LOCATION_STATE VARCHAR(50)) CHANGE_TRACKING = TRUE, comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';
create or replace TABLE dmm_customer_sample_db.sample_data.ITEMS (ITEM_NAME VARCHAR(200), ITEM_ID NUMBER(38,0), ITEM_DESCRIPTION VARCHAR(16777216), ITEM_PRICE FLOAT, ITEM_COST FLOAT)CHANGE_TRACKING = TRUE, comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';
create or replace TABLE dmm_customer_sample_db.sample_data.INVENTORIES (LOCATION_ID VARCHAR(80), ITEM_ID NUMBER(38,0), AMOUNT FLOAT, LAST_CHECKED_DATE TIMESTAMP_NTZ(9))CHANGE_TRACKING = TRUE, comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';

-- Insert Sample Locations
INSERT INTO dmm_customer_sample_db.sample_data.LOCATIONS
VALUES
('Warehouse A','1','Indianapolis','IN'),
('Warehouse B','2','Chicago','IL'),
('Warehouse C','3','Miami','FL'),
('Store 729','4','Los Angeles','CA'),
('Store 112','5','San Mateo','CA');

-- Insert Sample Items
INSERT INTO dmm_customer_sample_db.sample_data.ITEMS (ITEM_NAME, ITEM_ID, ITEM_DESCRIPTION, ITEM_PRICE, ITEM_COST)
VALUES
('Laptop', '1', 'A powerful laptop with the latest specs', 999.99, 799.99),
('Desktop PC', '2', 'High-performance desktop computer', 1299.99, 999.99),
('Monitor', '3', '27-inch Full HD monitor', 249.99, 199.99),
('Keyboard', '4', 'Mechanical gaming keyboard', 99.99, 69.99),
('Mouse', '5', 'Wireless ergonomic mouse', 49.99, 29.99),
('Headphones', '6', 'Noise-canceling headphones', 149.99, 99.99),
('External Hard Drive', '7', '2TB USB 3.0 External Hard Drive', 79.99, 59.99),
('SSD', '8', '500GB Solid State Drive', 99.99, 79.99),
('Router', '9', 'Dual-band Wi-Fi router', 79.99, 49.99),
('Printer', '10', 'Wireless all-in-one printer', 149.99, 99.99),
('Graphics Card', '11', 'Graphics RTB 9080', 699.99, 499.99),
('RAM', '12', '16GB DDR4 RAM', 79.99, 49.99),
('Webcam', '13', '1080p HD Webcam', 59.99, 39.99),
('USB Flash Drive', '14', '128GB USB 3.0 Flash Drive', 24.99, 14.99),
('Wireless Earbuds', '15', 'Bluetooth wireless earbuds', 79.99, 59.99),
('Software', '16', 'Office 24/7', 99.99, 79.99),
('Gaming Console', '17', 'Next-gen gaming console', 499.99, 399.99),
('Smartphone', '18', 'Latest smartphone model', 899.99, 699.99),
('Tablet', '19', '10-inch tablet with high-res display', 299.99, 199.99),
('Network Switch', '20', '8-port gigabit network switch', 49.99, 29.99);

-- Insert Sample Inventories
INSERT INTO dmm_customer_sample_db.sample_data.INVENTORIES (LOCATION_ID, ITEM_ID, AMOUNT, LAST_CHECKED_DATE)
VALUES
('1', '1', 10, '2024-05-01'),
('1', '2', 5, '2024-04-01'),
('1', '3', 20, '2024-03-01'),
('1', '4', 15, '2024-02-10'),
('1', '5', 30, '2023-05-01'),
('1', '6', 8, '2022-05-01'),
('1', '7', 12, '2024-04-11'),
('1', '8', 18, '2024-05-01'),
('1', '9', 10, '2024-05-01'),
('1', '10', 7, '2024-05-01'),
('1', '11', 3, '2024-05-01'),
('1', '12', 15, '2024-05-01'),
('1', '13', 25, '2024-05-01'),
('1', '14', 20, '2024-05-01'),
('1', '15', 10, '2024-05-01'),
('1', '16', 5, '2024-05-01'),
('1', '17', 4, '2024-05-01'),
('1', '18', 6, '2024-05-01'),
('1', '19', 8, '2024-05-01'),
('1', '20', 15, '2024-05-01'),
('2', '1', 8, '2024-05-01'),
('2', '2', 12, '2024-05-01'),
('2', '3', 25, '2024-05-01'),
('2', '4', 18, '2024-05-01'),
('2', '5', 22, '2024-05-01'),
('2', '6', 10, '2024-05-01'),
('2', '7', 15, '2024-05-01'),
('2', '8', 20, '2024-05-01'),
('2', '9', 12, '2024-05-01'),
('2', '10', 9, '2024-05-01'),
('2', '11', 5, '2024-05-01'),
('2', '12', 20, '2024-05-01'),
('3', '13', 30, '2024-05-01'),
('3', '14', 25, '2024-05-01'),
('3', '15', 15, '2024-05-01'),
('4', '16', 8, '2024-05-01'),
('4', '17', 6, '2024-05-01'),
('5', '18', 10, '2024-05-01'),
('5', '19', 12, '2024-05-01'),
('5', '20', 18, '2024-05-01');

create or replace TABLE dmm_customer_sample_db.sample_data.CLIENT (SOURCE VARCHAR(80), NATIVE_ID VARCHAR(80), NAME_PREFIX VARCHAR(50), FIRST_NAME VARCHAR(50), MIDDLE_NAME VARCHAR(50), LAST_NAME VARCHAR(50), MAIDEN_NAME VARCHAR(50), PERSON_SUFFIX VARCHAR(50), GENDER CHAR(1), FAKE_SSN VARCHAR(50), DOB DATE, DECEASED CHAR(1)) CHANGE_TRACKING = TRUE, comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';
create or replace TABLE dmm_customer_sample_db.sample_data.CLIENT_CONTACT_INFO (NATIVE_ID VARCHAR(80), PHONE_AREA_CODE VARCHAR(50), PHONE_NUMBER VARCHAR(50), EMAIL VARCHAR(50), STREET_ADDRESS VARCHAR(50), CITY VARCHAR(50), STATE VARCHAR(50), ZIPCODE VARCHAR(50)) CHANGE_TRACKING = TRUE, comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';

-- Insert Sample Customers
INSERT INTO dmm_customer_sample_db.sample_data.CLIENT
VALUES
('allscripts','b7cd8d39-fd6c-41a3-9336-56d806d8f103','Misc.', 'Dana','Jackie','Perez', 'Larson', 'Jr.', 'F', '222222222', '5/7/16', 'Y'),
('lab','625f6014-3437-4139-8168-fd83013dad74','Dr.', 'Ashley','Deborah','Thompson', 'Hernandez', 'DDS', 'F', '000000000','4/8/71', 'N'),
('epic','606866ed-0cc6-4bc4-ba03-339db6df0bcc','Mx.', 'Natalie','Frank','Monroe', 'Delgado', 'MD', 'O', '123456789','4/29/16', 'N'),
('cerner','991c541a-1b54-4a66-84b8-f8062d4a7168','Misc.', 'Susan','Gregory','Sharp', 'Hansen', 'Jr.', 'U', '987654321','1/12/97', 'Y'),
('','a51ee75a-a659-4de6-b69f-c288eb861e9e','Ms.', 'Steven','Jason','Washington', 'Taylor', 'III', 'O', '543210000', '4/12/89', 'Y');

-- Insert Sample Customers Contact Information
INSERT INTO dmm_customer_sample_db.sample_data.CLIENT_CONTACT_INFO
VALUES
('b7cd8d39-fd6c-41a3-9336-56d806d8f103','781', '3112915','wmartinez@example.org','1760 Boone Stream Suite 307', 'New Tiffanyville', 'Alabama', '23560-9688'),
('625f6014-3437-4139-8168-fd83013dad74','494', '3951822','braunhaley@example.net','9304 Garcia Heights Suite 479', 'Krystalview', 'Florida', '51593-9205'),
('606866ed-0cc6-4bc4-ba03-339db6df0bcc','598', '2388821','perezannette@example.net','4211 Joshua Forks Suite 811', 'East Tracy', 'California', '49505-6366'),
('991c541a-1b54-4a66-84b8-f8062d4a7168','351', '1941929','aaron41@example.com','0238 Aaron Vista Apt. 113', 'Port Derekberg', 'Oregon', '91333-8261'),
('a51ee75a-a659-4de6-b69f-c288eb861e9e','267', '8437843','kevingrimes@example.net','35047 Chandler Ford', 'Martinezmouth', 'Missouri', '00981-8780');

-- Test Validation Rules
INSERT INTO data_model_mapper_app_package.admin.validation_rule(
TARGET_TABLE,
RULE_NAME,
VALIDATION_TYPE ,
VALIDATION_DESCRIP,
COLUMN_NAMES,
OPERATOR,
VALIDATION_PARAMS,
CUSTOM_SQL_CONDITION 
) 
SELECT 'Mock_Data__v1__CUSTOMER', 'Deceased in (Y, N)', 'DOMAIN_OF_VALUES', 'DECEASED MUST BE IN (Y, N)' , ARRAY_CONSTRUCT('DECEASED'), 'IN', PARSE_JSON('{"Conditions": ["Y", "N"]}'), NULL
UNION ALL 
SELECT 'Mock_Data__v1__CUSTOMER', 'Native ID Length', 'IS_LENGTH', 'NATIVE_ID LENGTH MUST = 36' , ARRAY_CONSTRUCT('NATIVE_ID'), '=', PARSE_JSON('{"Conditions": 36}'), NULL
UNION ALL
SELECT 'Mock_Data__v1__CUSTOMER', 'Count Unique Records', 'COUNT','NATIVE_ID MUST CONTAIN ONLY UNIQUE VALUES', ARRAY_CONSTRUCT('NATIVE_ID'), '', PARSE_JSON('{"Conditions": 0}'), NULL
UNION ALL
SELECT 'Mock_Data__v1__CUSTOMER_INFO', 'Area Code Demographic', 'RANGE', 'PHONE_AREA_CODE MUST BE BETWEEN 185 AND 400', ARRAY_CONSTRUCT('PHONE_AREA_CODE'), 'BETWEEN', PARSE_JSON('{"Conditions": [185, 400]}'), NULL
UNION ALL
SELECT 'Mock_Data__v1__CUSTOMER_INFO', 'Data Type Check', 'DATA_TYPE', 'ZIPCODE MUST BE STORED AS TYPE SPOCK',ARRAY_CONSTRUCT('ZIPCODE'), 'IS', PARSE_JSON('{"Conditions": "SPOCK"}'), NULL
UNION ALL
SELECT 'Mock_Data__v1__CUSTOMER', 'Source and Native ID populated', 'REQD_COLS_POPULATED', 'Source and ID must be not null and not empty' , ARRAY_CONSTRUCT('SOURCE', 'NATIVE_ID'), NULL, NULL, NULL
UNION ALL
SELECT 'Mock_Data__v1__CUSTOMER_INFO', 'Phone Number Format', 'REGEX', 'PHONE_NUMBER MUST BE IN THE FORMAT XXX-XXX-XXXX', ARRAY_CONSTRUCT('PHONE_NUMBER'), NULL, PARSE_JSON('{"Conditions": "^[0-9]{3}-[0-9]{3}-[0-9]{4}$"}'), NULL;


Create a version and install the app

In [ ]:
-- use role accountadmin;
-- call system$wait(5);
-- use role data_model_mapper_role;
-- call system$wait(5);
use role accountadmin;
call system$wait(5);

/* create application package version using code files */
ALTER APPLICATION PACKAGE data_model_mapper_app_package
    ADD VERSION Version1 USING '@data_model_mapper_deployment_db.code.files_v1_p0';

ALTER APPLICATION PACKAGE data_model_mapper_app_package
  SET DEFAULT RELEASE DIRECTIVE
  VERSION = Version1
  PATCH = 0;    

/* test creating the app locally */
CREATE APPLICATION data_model_mapper_app FROM APPLICATION PACKAGE data_model_mapper_app_package;

/* requires that the app was installed as data_model_mapper_app in the previous step */
grant application role data_model_mapper_app.dmm_consumer_app_role to role data_model_mapper_role;

/* nonprod dmm application role */
grant application role data_model_mapper_app.dmm_demo_app_role to role data_model_mapper_role;

/* grant required app permissions */
use role accountadmin;
call system$wait(5);
grant create database on account to application data_model_mapper_app;
grant execute task on account to application data_model_mapper_app;
grant create warehouse on account to application data_model_mapper_app;

/* initializes the app by creating the outside database - required for operation */
call data_model_mapper_app.modeling.initialize_application();

/* grant app access to demo data */
grant usage on database dmm_customer_sample_db to application data_model_mapper_app;
grant usage on schema dmm_customer_sample_db.sample_data to application data_model_mapper_app;
grant select on all tables in schema dmm_customer_sample_db.sample_data to application data_model_mapper_app;

/* grant permissions on app's share db to data_model_mapper_role */
use role accountadmin;
call system$wait(5);

/* Make required grants */
grant all privileges on database data_model_mapper_share_db to role data_model_mapper_role with grant option;
grant all privileges on schema data_model_mapper_share_db.configuration to role data_model_mapper_role with grant option;
grant all privileges on schema data_model_mapper_share_db.modeled to role data_model_mapper_role with grant option;
grant all privileges on schema data_model_mapper_share_db.mapped to role data_model_mapper_role with grant option;
grant all privileges on schema data_model_mapper_share_db.utility to role data_model_mapper_role with grant option;
grant all privileges on schema data_model_mapper_share_db.validated to role data_model_mapper_role with grant option;

grant all privileges on all tables in schema data_model_mapper_share_db.configuration to role data_model_mapper_role;
grant all privileges on future tables in schema data_model_mapper_share_db.configuration to role data_model_mapper_role;
grant all privileges on all tables in schema data_model_mapper_share_db.modeled to role data_model_mapper_role;
grant all privileges on future tables in schema data_model_mapper_share_db.modeled to role data_model_mapper_role;
grant all privileges on all views in schema data_model_mapper_share_db.mapped to role data_model_mapper_role;
grant all privileges on future views in schema data_model_mapper_share_db.mapped to role data_model_mapper_role;
grant all privileges on all tables in schema data_model_mapper_share_db.validated to role data_model_mapper_role;
grant all privileges on future tables in schema data_model_mapper_share_db.validated to role data_model_mapper_role;
grant all privileges on all functions in schema data_model_mapper_share_db.utility to role data_model_mapper_role;
grant all privileges on all procedures in schema data_model_mapper_share_db.utility to role data_model_mapper_role;
grant ownership on procedure data_model_mapper_share_db.utility.share_views() to role data_model_mapper_role revoke current grants;

use role data_model_mapper_role;
call system$wait(5);


/* sharing testing - will be performed by the consumer through Streamlit */
/* note - the Listing API is in private preview and only for non-production use cases */
/* the native app can help users walk through using the GUI, which is GA, and the Listing API added later */
drop share if exists data_model_mapper_share;
create or replace share data_model_mapper_share;
grant usage on database data_model_mapper_share_db to share data_model_mapper_share;
grant usage on schema data_model_mapper_share_db.configuration to share data_model_mapper_share;
grant usage on schema data_model_mapper_share_db.mapped to share data_model_mapper_share;
grant usage on schema data_model_mapper_share_db.validated to share data_model_mapper_share;


grant select on table data_model_mapper_share_db.configuration.source_collection to share data_model_mapper_share;
grant select on table data_model_mapper_share_db.configuration.source_collection_filter_condition to share data_model_mapper_share;
grant select on table data_model_mapper_share_db.configuration.source_entity to share data_model_mapper_share;
grant select on table data_model_mapper_share_db.configuration.source_entity_join_condition to share data_model_mapper_share;
grant select on table data_model_mapper_share_db.configuration.source_entity_attribute to share data_model_mapper_share;
grant select on table data_model_mapper_share_db.configuration.source_to_target_mapping to share data_model_mapper_share;
grant select on table data_model_mapper_share_db.validated.validation_log to share data_model_mapper_share;

/* call this periodically to add new views to the share */
call data_model_mapper_share_db.utility.share_views();

 
/* FOR NON-PRODUCTION - feel free to use the Listing API to automate during development, but the Listing API is still in preview */
/* 
create listing data_model_mapper_app_share in data exchange SNOWFLAKE_DATA_MARKETPLACE
for share data_model_mapper_share as
$$
 title: "Data Mapping App Share"
 description: "The shareback from the Data Mapper App"
 terms_of_service:
   type: "OFFLINE"
 auto_fulfillment:
   refresh_schedule: "10 MINUTE"
   refresh_type: "FULL_DATABASE"
 targets:
   accounts: ["ORG_NAME.ACCOUNT_NAME"]
$$;

alter listing  data_model_mapper_app_share set state = published;

show listings like 'data_model_mapper_app_share' in data exchange snowflake_data_marketplace;

*/